In [1]:
import ROOT
import array as array
from array import array 
import os
import glob
import ROOT
from ROOT import TH1D, TH2D, TTree,TFile, TUnfoldSys, TUnfoldDensity, TUnfold, TDirectoryFile
from random import gauss
import numpy as np
import root_numpy as rtnpy
from root_numpy import *
import h5py
#from awkward import JaggedArray, Table
import matplotlib.pyplot as plt
import seaborn as sns

Welcome to JupyROOT 6.18/00


In [2]:
%jsroot on

In [3]:
def getListOfFiles(dirName, histos=False):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    #To ensure event counts don't come up against  a histogram file and rather just the nanoskim
    if histos==False: listOfFile[:] = [x for x in listOfFile if "histograms" not in x]
    #print (listOfFile)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):            
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

#print (getListOfFiles('/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/jetObservables_Skimmer_UL17v2_TTToSemileptonic_MiniAODv2_V2/210502_173740/0000/'))

In [4]:
nano_directories_17 = [
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_233932/0000/',
    
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_232025/0000/', 
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_232025/0001/',
    
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_233421/0000/', 
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_233421/0001/', 
    
     
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_s-channel_4f_leptonDecays_TuneCP5_13TeV-amcatnlo-pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_232435/0000/',
    
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_t-channel_top_5f_InclusiveDecays_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_233252/0000/',
    
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_t-channel_antitop_5f_InclusiveDecays_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_233556/0000/',
    
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_232040/0000/',
    
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_234008/0000/',
    
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_232318/0000/', 
    
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt-170to300_MuEnrichedPt5_TuneCP5_13TeV_pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_233420/0000/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt-300to470_MuEnrichedPt5_TuneCP5_13TeV_pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_233930/0000/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt-470to600_MuEnrichedPt5_TuneCP5_13TeV_pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_232431/0000/',  
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt-600to800_MuEnrichedPt5_TuneCP5_13TeV_pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_235309/0000/',  
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt-800to1000_MuEnrichedPt5_TuneCP5_13TeV_pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_234432/0000/',  
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/QCD_Pt-1000toInf_MuEnrichedPt5_TuneCP5_13TeV_pythia8/RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210610_234903/0000/',  
    
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/WW_TuneCP5_13TeV-pythia8/RunIISummer20UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210615_150121/0000/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/WZ_TuneCP5_13TeV-pythia8/RunIISummer20UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210615_150121/0000/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ZZ_TuneCP5_13TeV-pythia8/RunIISummer20UL17PFNanoAOD_jetObservables_Skimmer_PreFinal/210615_150120/0000/'
    
]

In [5]:
nano_directories_18= [
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210611_133039/0000/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210611_133039/0001/',

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210611_132349/0000/', 
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210611_132349/0001/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210611_132349/0002/',
    
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210611_132828/0000/', 
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210611_132828/0001/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210611_132828/0002/',
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210611_132828/0003/',
    
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_s-channel_4f_leptonDecays_TuneCP5_13TeV-amcatnlo-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210611_124952/0000/',

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_t-channel_top_5f_InclusiveDecays_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210611_134341/0000/',

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_t-channel_antitop_5f_InclusiveDecays_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210611_134550/0000/',

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_tW_top_5f_NoFullyHadronicDecays_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210611_134800/0000/',

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ST_tW_antitop_5f_NoFullyHadronicDecays_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210611_145536/0000/',

    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210611_132348/0000/',
    
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/WW_TuneCP5_13TeV-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210615_150117/0000/',
    
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/WZ_TuneCP5_13TeV-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210615_150119/0000/',
    
    '/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ZZ_TuneCP5_13TeV-pythia8/RunIISummer19UL18PFNanoAOD_jetObservables_Skimmer_PreFinal/210615_150203/0000/',
    
    
    ]

### Get list of nanoskim files to run over and sum gen event weights (ie, n_events) for a given samples

### UL 2017

In [36]:
for d in nano_directories_17:
    print ("\n", d.split("/",11)[9:11], ", # of nano skim files:", len(getListOfFiles(d)), "\n")
    genEventCount = 0
    genEventSumw = 0
    genEventSumw2 = 0
    sums=0.
    for filepath in getListOfFiles(d):
        #print (filepath)
        if "SingleMuon" in filepath: continue
        else:
            f = ROOT.TFile(filepath)
            #tE = f.Get("Events")
            tR = f.Get("Runs")
            arrays = tree2array( tR, branches=[ 'genEventSumw', ])
            genEventSumw+=sum(arrays['genEventSumw'])
             
            f.Close()
    print ("# of events = %f"%(genEventSumw))


 ['TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8', 'RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal'] , # of nano skim files: 801 

# of events = 4759777104.469413

 ['TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8', 'RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal'] , # of nano skim files: 999 

# of events = 24408467409.655987

 ['TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8', 'RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal'] , # of nano skim files: 324 

# of events = 7984320068.252007

 ['TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8', 'RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal'] , # of nano skim files: 999 

# of events = 162503781136.934235

 ['TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8', 'RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal'] , # of nano skim files: 766 

# of events = 124126008674.833481

 ['ST_s-channel_4f_leptonDecays_TuneCP5_13TeV-amcatnlo-pythia8', 'RunIISummer19UL17PFNanoAOD_jetObservables_Skimmer_PreFinal'] , 

In [6]:
dict_MCscaling_UL17 = {
    'TTJets': ["722.8", 286629789811.7677, #xsdb
               '../TTJets_amcatnloFXFX-pythia8_2017UL.root',
              ],
    'TTToSemiLeptonic': ["365.34", 32392787477.907993,
                         '../TTToSemileptonic_powheg_pythia8_2017UL.root',
                        ],
    'TTTo2L2Nu': ["88.29 ", 4759777104.469413, 
                  '../TTTo2L2Nu_powheg_pythia8_2017UL.root',
                 ],
    'WJetsToLNu': ["5.368e+04", 9949891586212.455078, 
                   '../WJetsToLNu_madgraphMLM_pythia8_2017UL.root',
                  ],
    'ST_s-channel': ["3.549e+00", 70439807.475972, 
                     '../ST_s-channel_amcatnlo_pythia8_2017UL.root',
                    ],
    'ST_t-channel_top': ["1.197e+02", 656310119.274000, 
                         '../ST_t-channel_top_powheg_pythia8_2017UL.root'
                        ],
    'ST_t-channel_antitop': ["7.174e+01", 265124234.844000,  #xsdb says 82.52
                            '../ST_t-channel_antitop_powheg_pythia8_2017UL.root'
                            ],
    'ST_tW_top': ["3.245e+01", 331239851.967700, 
                  '../ST_tW_top_powheg_pythia8_2017UL.root',
                 ],
    'ST_tW_antitop': ["3.251e+01", 303960792.435600, 
                     '../ST_tW_antitop_powheg_pythia8_2017UL.root',
                     ],
    'QCD_Pt_170to300': ["7055.0", 36027673.000000, 
                        '../QCD_Muen_Pt_170to300_pythia8_2017UL.root',
                       ],
    'QCD_Pt_300to470': ["619.3", 494796.072880, 
                        '../QCD_Muen_Pt_300to470_pythia8_2017UL.root',   
                       ],
    'QCD_Pt_470to600': ["59.24", 517383.713837, 
                        '../QCD_Muen_Pt_470to600_pythia8_2017UL.root',
                       ],
    'QCD_Pt_600to800': ["18.21", 17318812.571925, 
                        '../QCD_Muen_Pt_600to800_pythia8_2017UL.root',
                       ],
    'QCD_Pt_800to1000': ["2.597e+01", 16962615.000000, 
                         '../QCD_Muen_Pt_800to1000_pythia8_2017UL.root'    
                        ],
    'QCD_Pt_1000toInf': ["1.078", 14642553.000000, 
                          '../QCD_Muen_Pt_1000toInf_pythia8_2017UL.root',
                         ],
    'WW': ["7.577e+01", 15634116.199514, 
          '../WW_pythia8_2017UL.root'  
          ],
    'WZ': ["1.21e+00", 7889000.000000, 
           '../WZ_pythia8_2017UL.root'    
          ],
    'ZZ': ["2.748e+00", 2706000.000000, 
           '../ZZ_pythia8_2017UL.root'
          ]
    }


In [7]:
lumi=41472.052#4653.71+9289.53+4247.68+9312.83+11758.83
print (lumi)

samples = list(dict_MCscaling_UL17.keys())
dict_items = list(dict_MCscaling_UL17.items())
xsecs = np.array(list(float(dict_items[k][1][0]) for k in np.arange(0,len(dict_items)))) 
nevts = np.array(list(float(dict_items[k][1][1]) for k in np.arange(0,len(dict_items)))) 
fpathsMC = np.array(list((dict_items[k][1][2]) for k in np.arange(0,len(dict_items))))
fpathData = '../SingleMuonAll_2017UL.root'
for i in range(0,len(samples)):
    print (i, xsecs[i], samples[i], nevts[i], fpathsMC[i])

41472.052
0 722.8 TTJets 286629789811.7677 ../TTJets_amcatnloFXFX-pythia8_2017UL.root
1 365.34 TTToSemiLeptonic 32392787477.907993 ../TTToSemileptonic_powheg_pythia8_2017UL.root
2 88.29 TTTo2L2Nu 4759777104.469413 ../TTTo2L2Nu_powheg_pythia8_2017UL.root
3 53680.0 WJetsToLNu 9949891586212.455 ../WJetsToLNu_madgraphMLM_pythia8_2017UL.root
4 3.549 ST_s-channel 70439807.475972 ../ST_s-channel_amcatnlo_pythia8_2017UL.root
5 119.7 ST_t-channel_top 656310119.274 ../ST_t-channel_top_powheg_pythia8_2017UL.root
6 71.74 ST_t-channel_antitop 265124234.844 ../ST_t-channel_antitop_powheg_pythia8_2017UL.root
7 32.45 ST_tW_top 331239851.9677 ../ST_tW_top_powheg_pythia8_2017UL.root
8 32.51 ST_tW_antitop 303960792.4356 ../ST_tW_antitop_powheg_pythia8_2017UL.root
9 7055.0 QCD_Pt_170to300 36027673.0 ../QCD_Muen_Pt_170to300_pythia8_2017UL.root
10 619.3 QCD_Pt_300to470 494796.07288 ../QCD_Muen_Pt_300to470_pythia8_2017UL.root
11 59.24 QCD_Pt_470to600 517383.713837 ../QCD_Muen_Pt_470to600_pythia8_2017UL.root


In [8]:
print (len((dict_MCscaling_UL17.keys())))

18


In [9]:
import array
from array import array

In [10]:
def Bin(Range):
    bins=np.array([x for x in np.linspace(num=Range[0]+1,start=Range[1],stop=Range[2])])
    return bins

def getRebinnedRescaledHist(hist_name, file_name, xrange, lumi, xsec, nevt, isMC=True, rebin=True):
    #if not(sample_name in file_name): 
    #    return "File either not available or needs to be renamed to be compatible with this (poorly,) hard-coded piece of software..."
    
    FILE = ROOT.TFile.Open(file_name,'read')
    tree = FILE.Get('jetObservables')
    hist = tree.Get(hist_name)
    hist.SetDirectory(0)
    
    ROOT.TH1.AddDirectory(ROOT.kFALSE);
    
    bins = array('d', np.array(Bin(xrange)))
    
    if rebin: newHist = ROOT.TH1F(hist.Rebin(len(bins)-1, "%s_rebin"%hist_name, bins))
    else: newHist = hist.Clone()
    #newHist.SetStats(ROOT.kFALSE)
    if isMC==True: newHist.Scale(lumi*xsec/nevt)
    
    return newHist

In [11]:
Hists_WSel=np.array([
    'recoJet_pt_nom_WSel', 'recoJet_mass_nom_WSel', #'recoJetTau21_nom_WSel', 'recoJetTau32_nom_WSel',
    'recoJet_tau_0p5_1_nom_WSel', 'recoJet_tau_1_1_nom_WSel', 'recoJet_tau_2_1_nom_WSel',
    'recoJet_tau_0p5_2_nom_WSel', 'recoJet_tau_1_2_nom_WSel', 'recoJet_tau_2_2_nom_WSel',
    'recoJet_tau_0p5_3_nom_WSel', 'recoJet_tau_1_3_nom_WSel', 'recoJet_tau_2_3_nom_WSel',
    'recoJet_tau_0p5_4_nom_WSel', 'recoJet_tau_1_4_nom_WSel', 'recoJet_tau_2_4_nom_WSel',
    'nPVs_WSel','nleps_WSel',
    'muons_pt_WSel','muons_eta_WSel','muons_phi_WSel',
    'nAK8jets_WSel','AK8jets_pt_WSel','AK8jets_eta_WSel','AK8jets_phi_WSel','AK8jets_mass_WSel',
    'nAK4jets_WSel','AK4jets_pt_WSel','AK4jets_eta_WSel','AK4jets_phi_WSel','AK4jets_mass_WSel',
    'METPt_WSel','HT_WSel'])
#Objects_WSel=np.concatenate((Objects1_WSel,Objects3_WSel),axis=0)

Histnames_WSel=np.array([
    ['Events/(50 GeV)','P_{t}','P_{t}/GeV'],['Events/(10 GeV)','Softdrop Mass','Softdrop Mass/GeV'],#['Events/0.05','#tau_{21}','#tau_{21}'],['Events/0.05','#tau_{32}','#tau_{32}'],
    ['Events/0.03','#tau_{1}^{(0.5)}','#tau_{1}^{(0.5)}'],['Events/0.02','#tau_{1}^{(1)}','#tau_{1}^{(1)}'],['Events/0.02','#tau_{1}^{(2)}','#tau_{1}^{(2)}'],
    ['Events/0.03','#tau_{2}^{(0.5)}','#tau_{2}^{(0.5)}'],['Events/0.02','#tau_{2}^{(1)}','#tau_{2}^{(1)}'],['Events/0.01','#tau_{2}^{(2)}','#tau_{2}^{(2)}'],
    ['Events/0.02','#tau_{3}^{(0.5)}','#tau_{3}^{(0.5)}'],['Events/0.02','#tau_{3}^{(1)}','#tau_{3}^{(1)}'],['Events/0.01','#tau_{3}^{(2)}','#tau_{3}^{(2)}'],
    ['Events/0.02','#tau_{4}^{(0.5)}','#tau_{4}^{(0.5)}'],['Events/0.01','#tau_{4}^{(1)}','#tau_{4}^{(1)}'],['Events/0.01','#tau_{4}^{(2)}','#tau_{4}^{(2)}'],
    ['Events/2','Number of PV','Number of PV'],['Events','Number of Leptons','Number of Leptons'],
    ['Events/(20 GeV)','P_{t} of Muons','P_{t} of Muons/GeV'],['Events/0.4','#eta of Muons','#eta of Muons'],['Events/(#pi/10)','#phi of Muons','#phi of Muons'],
    ['Events','Number of AK8Jets','Number of AK8Jets'],['Events/(60 GeV)','P_{t} of AK8Jets','P_{t} of AK8Jets/GeV'],['Events/0.4','#eta of AK8Jets','#eta of AK8Jets'],['Events/(#pi/10)','#phi of AK8Jets','#phi of AK8Jets'],['Events/(10 GeV)','Mass of AK8Jets','Mass of AK8Jets/GeV'],
    ['Events','Number of AK4Jets','Number of AK4Jets'],['Events/(40 GeV)','P_{t} of AK4Jets','P_{t} of AK4Jets/GeV'],['Events/0.4','#eta of AK4Jets','#eta of AK4Jets'],['Events/(#pi/10)','#phi of AK4Jets','#phi of AK4Jets'],['Events/(20 GeV)','Mass of AK4Jets','Mass of AK4Jets/GeV'],
    ['Events/(20 GeV)','MET P_{t}','MET P_{t}/GeV'],['Events/(100 GeV)','HT','HT/GeV']])
Range_WSel=[
    [20,0,1000], [11,50,160],#[20,0,1],[20,0,1],
    [30,0,0.9],[35,0,0.7],[20,0,0.4],
    [20,0,0.6],[20,0,0.4],[20,0,0.2],
    [25,0,0.5],[20,0,0.4],[20,0,0.1],
    [20,0,0.4],[20,0,0.2],[20,0,0.1],  
    [35,0,70],[4,0,4],
    [20,0,400],[20,-4,4],[20,-3.14259, 3.14159],
    [8,0,8],[20,0,1200],[20,-4,4],[20,-3.14259, 3.14159],[15,0,150],
    [8,0,8],[20,0,800],[20,-4,4],[20,-3.14259, 3.14159],[20,0,400],
    [30,0,600],[20,0,2000]
]
print (len(Hists_WSel),len(Histnames_WSel),len(Range_WSel))

31 31 31


In [12]:
print (Histnames_WSel[0][0]
      )

Events/(50 GeV)


In [13]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.12

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [14]:
c = np.zeros(len(Range_WSel), ROOT.TCanvas)
l = np.zeros(len(Range_WSel), ROOT.TLegend)

H_ref = 720; 
W_ref = 1024; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.97
y1_l = 0.85

dx_l = 0.35
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_WSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_WSel)):
    
    print (i, Hists_WSel[i], fpathData, Range_WSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],0,0,W,H)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    c[i].SetLeftMargin( L/W )
    c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 0.95, -1,0,0);
    pad1.SetBottomMargin(0.); #// Upper and lower plot are joined
    #pad1.SetGridx();         #// Vertical grid
    pad1.SetBorderMode(0)
    pad1.SetBorderSize(0)
    pad1.SetFrameLineWidth(0)
    pad1.SetFrameBorderMode(0)
    pad1.Draw()           #// Draw the upper pad: pad1
    pad1.cd()
    Pad1.append(pad1)
    
    
    

    MC = ROOT.THStack("MC", Hists_WSel[i])
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1]))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetFillColor(ROOT.kGreen+3)
    
    TTDileptonic = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[2], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[2], nevt=nevts[2]))
    TTDileptonic.SetLineColor(ROOT.kRed-9)
    TTDileptonic.SetFillColor(ROOT.kRed-9)
    
    WJets = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[3], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[3], nevt=nevts[3]))
    WJets.SetLineColor(ROOT.kCyan-6)
    WJets.SetFillColor(ROOT.kCyan-6)
    
    SingleTop = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(4,9):
        #print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                               file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                               lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        SingleTop.Add(hist)
        
    SingleTop.SetLineColor(ROOT.kAzure+8)
    SingleTop.SetFillColor(ROOT.kAzure+8)
    
    QCD = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(9,14):
        #print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                         file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                         lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        QCD.Add(hist)
    QCD.SetLineColor(ROOT.kPink-1)
    QCD.SetFillColor(ROOT.kPink-1)
    
    VV = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(14,17):
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                        file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                        lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        VV.Add(hist)
    VV.SetLineColor(ROOT.kGray+2)
    VV.SetFillColor(ROOT.kGray+2)
    
    #VV.GetYaxis().SetRangeUser(-0.5, 1.25*TTSemileptonic.GetMaximum())
    MC.Add(VV)
    MC.Add(SingleTop)
    MC.Add(QCD)
    MC.Add(WJets)
    MC.Add(TTDileptonic)
    MC.Add(TTSemileptonic)
    dummy = MC.GetHists().At(0)
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    dummy.Draw()  
    dummy.GetYaxis().SetTitle(Histnames_WSel[i][0])
    dummy.GetXaxis().SetTitle(Histnames_WSel[i][2]) 
    print(Histnames_WSel[i][2])
    dummy.GetYaxis().SetRangeUser(-0.5, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    MC.Draw("hist e")
    
    c[i].Update()
    MC.Draw("histsame e")
    Data_SingleMuonUL17 = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathData, xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], isMC=False))
    Data_SingleMuonUL17.SetMarkerSize(1.5)
    Data_SingleMuonUL17.SetMarkerColor(ROOT.kBlack)
    Data_SingleMuonUL17.SetMarkerStyle(20)
    Data_SingleMuonUL17.SetLineColor(ROOT.kBlack)
    Data_SingleMuonUL17.Draw("epsamex0")
    
    
    CMS_lumi.CMS_lumi(c[i], iPeriod, iPos)
    c[i].cd()
    c[i].Update()
    c[i].RedrawAxis()
    frame = c[i].GetFrame()
    frame.Draw()

    pad2 = ROOT.TPad("pad2","pad2",0,0.0,1,0.3, -1,0,0)
    pad2.SetTopMargin(0)
    pad2.SetGridx()
    pad2.SetGridy()
    pad2.Draw()
    pad2.cd()
    pad2.SetBottomMargin(0.3)
    Pad2.append(pad2)

    ratio = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    ratio.GetXaxis().SetTitle(Histnames_WSel[i][2])
    ratio.Add(Data_SingleMuonUL17)
    ratio.GetYaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetXaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetYaxis().SetRangeUser(0.3,1.7)
    
    mc_rt = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    mc_rt.Add(TTSemileptonic)
    mc_rt.Add(TTDileptonic)
    mc_rt.Add(SingleTop)
    mc_rt.Add(QCD)
    mc_rt.Add(VV)
    mc_rt.Add(WJets)
    
    ratio.GetYaxis().SetTitle("#frac{Data}{MC}")
    ratio.Sumw2()
    ratio.SetStats(0)
    ratio.Divide(mc_rt)
    ratio.GetYaxis().SetLabelSize(0.09)
    ratio.GetXaxis().SetLabelSize(0.09)
    ratio.SetMarkerColor(ROOT.kBlack)
    ratio.SetMarkerStyle(20)
    ratio.SetLineColor(ROOT.kBlack)
    ratio.GetYaxis().SetTitleSize(0.12)
    ratio.GetYaxis().SetTitleOffset(0.4)
    ratio.GetXaxis().SetTitleSize(0.12)
    ratio.GetXaxis().SetTitleOffset(0.85)
    Ratio.append(ratio)
    #ratio.SetAxisRange(50,150)

    ratio.Draw("EP")
    c[i].cd()

    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} semileptonic (Powheg)","f")
    l[i].AddEntry(TTDileptonic,"t#bar{t} dileptonic","f")
    l[i].AddEntry(SingleTop,"Single top","f")
    l[i].AddEntry(WJets,"W+Jets","f")
    l[i].AddEntry(QCD,"QCD multijets","f")
    l[i].AddEntry(VV,"Diboson","f")
    l[i].AddEntry(Data_SingleMuonUL17,"Data UL 2017","lep")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("WSel/UL17/"+Hists_WSel[i]+"_powheg_p8_UL17.png")
    c[i].Close()

0 recoJet_pt_nom_WSel ../SingleMuonAll_2017UL.root [20, 0, 1000] 41472.052 365.34 32392787477.907993
P_{t}/GeV
1 recoJet_mass_nom_WSel ../SingleMuonAll_2017UL.root [11, 50, 160] 41472.052 365.34 32392787477.907993
Softdrop Mass/GeV
2 recoJet_tau_0p5_1_nom_WSel ../SingleMuonAll_2017UL.root [30, 0, 0.9] 41472.052 365.34 32392787477.907993
#tau_{1}^{(0.5)}
3 recoJet_tau_1_1_nom_WSel ../SingleMuonAll_2017UL.root [35, 0, 0.7] 41472.052 365.34 32392787477.907993
#tau_{1}^{(1)}
4 recoJet_tau_2_1_nom_WSel ../SingleMuonAll_2017UL.root [20, 0, 0.4] 41472.052 365.34 32392787477.907993
#tau_{1}^{(2)}
5 recoJet_tau_0p5_2_nom_WSel ../SingleMuonAll_2017UL.root [20, 0, 0.6] 41472.052 365.34 32392787477.907993
#tau_{2}^{(0.5)}
6 recoJet_tau_1_2_nom_WSel ../SingleMuonAll_2017UL.root [20, 0, 0.4] 41472.052 365.34 32392787477.907993
#tau_{2}^{(1)}
7 recoJet_tau_2_2_nom_WSel ../SingleMuonAll_2017UL.root [20, 0, 0.2] 41472.052 365.34 32392787477.907993
#tau_{2}^{(2)}
8 recoJet_tau_0p5_3_nom_WSel ../SingleMu

Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL17/recoJet_pt_nom_WSel_powheg_p8_UL17.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL17/recoJet_mass_nom_WSel_powheg_p8_UL17.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL17/recoJet_tau_0p5_1_nom_WSel_powheg_p8_UL17.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL17/recoJet_tau_1_1_nom_WSel_powheg_p8_UL17.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL17/recoJet_tau_2_1_nom_WSel_powheg_p8_UL

In [15]:
c = np.zeros(len(Range_WSel), ROOT.TCanvas)
l = np.zeros(len(Range_WSel), ROOT.TLegend)

H_ref = 720; 
W_ref = 1024; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.97
y1_l = 0.85

dx_l = 0.35
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_WSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_WSel)):
    
    print (i, Hists_WSel[i], fpathData, Range_WSel[i], lumi, xsecs[0], nevts[0])

    c[i] = ROOT.TCanvas(CName[i],CName[i],0,0,W,H)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    c[i].SetLeftMargin( L/W )
    c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 0.95, -1,0,0);
    pad1.SetBottomMargin(0.); #// Upper and lower plot are joined
    #pad1.SetGridx();         #// Vertical grid
    pad1.SetBorderMode(0)
    pad1.SetBorderSize(0)
    pad1.SetFrameLineWidth(0)
    pad1.SetFrameBorderMode(0)
    pad1.Draw()           #// Draw the upper pad: pad1
    pad1.cd()
    Pad1.append(pad1)
    
    
    

    MC = ROOT.THStack("MC", Hists_WSel[i])
    
    TTJets = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[0], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[0], nevt=nevts[0]))
    TTJets.SetLineColor(ROOT.kGreen+3)
    TTJets.SetFillColor(ROOT.kGreen+3)
    
    
    WJets = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[3], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[3], nevt=nevts[3]))
    WJets.SetLineColor(ROOT.kCyan-6)
    WJets.SetFillColor(ROOT.kCyan-6)
    
    SingleTop = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(4,9):
        print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                               file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                               lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        SingleTop.Add(hist)
        
    SingleTop.SetLineColor(ROOT.kAzure+8)
    SingleTop.SetFillColor(ROOT.kAzure+8)
    
    QCD = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(9,14):
        print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                         file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                         lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        QCD.Add(hist)
    QCD.SetLineColor(ROOT.kPink-1)
    QCD.SetFillColor(ROOT.kPink-1)
    
    VV = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(14,17):
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                        file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                        lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        VV.Add(hist)
    VV.SetLineColor(ROOT.kGray+2)
    VV.SetFillColor(ROOT.kGray+2)
    
    #VV.GetYaxis().SetRangeUser(-0.5, 1.25*TTJets.GetMaximum())
    MC.Add(VV)
    MC.Add(SingleTop)
    MC.Add(QCD)
    MC.Add(WJets)
    MC.Add(TTJets)
    dummy = MC.GetHists().At(0)
    #dummy.SetAxisRange(0., TTJets.GetMaximum()*1.5)
    dummy.Draw()  
    dummy.GetYaxis().SetTitle(Histnames_WSel[i][0])
    dummy.GetXaxis().SetTitle(Histnames_WSel[i][2]) 
    print(Histnames_WSel[i][2])
    dummy.GetYaxis().SetRangeUser(-0.5, 2.5*TTJets.GetMaximum())
    c[i].Update()
    MC.Draw("hist e")
    
    c[i].Update()
    MC.Draw("histsame e")
    Data_SingleMuonUL17 = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathData, xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[0], nevt=nevts[0], isMC=False))
    Data_SingleMuonUL17.SetMarkerSize(1.5)
    Data_SingleMuonUL17.SetMarkerColor(ROOT.kBlack)
    Data_SingleMuonUL17.SetMarkerStyle(20)
    Data_SingleMuonUL17.SetLineColor(ROOT.kBlack)
    Data_SingleMuonUL17.Draw("epsamex0")
    
    
    CMS_lumi.CMS_lumi(c[i], iPeriod, iPos)
    c[i].cd()
    c[i].Update()
    c[i].RedrawAxis()
    frame = c[i].GetFrame()
    frame.Draw()

    pad2 = ROOT.TPad("pad2","pad2",0,0.0,1,0.3, -1,0,0)
    pad2.SetTopMargin(0)
    pad2.SetGridx()
    pad2.SetGridy()
    pad2.Draw()
    pad2.cd()
    pad2.SetBottomMargin(0.3)
    Pad2.append(pad2)

    ratio = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    ratio.GetXaxis().SetTitle(Histnames_WSel[i][2])
    ratio.Add(Data_SingleMuonUL17)
    ratio.GetYaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetXaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetYaxis().SetRangeUser(0.3,1.7)
    
    mc_rt = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    mc_rt.Add(TTJets)
    mc_rt.Add(SingleTop)
    mc_rt.Add(QCD)
    mc_rt.Add(VV)
    mc_rt.Add(WJets)
    
    ratio.GetYaxis().SetTitle("#frac{Data}{MC}")
    ratio.Sumw2()
    ratio.SetStats(0)
    ratio.Divide(mc_rt)
    ratio.GetYaxis().SetLabelSize(0.09)
    ratio.GetXaxis().SetLabelSize(0.09)
    ratio.SetMarkerColor(ROOT.kBlack)
    ratio.SetMarkerStyle(20)
    ratio.SetLineColor(ROOT.kBlack)
    ratio.GetYaxis().SetTitleSize(0.12)
    ratio.GetYaxis().SetTitleOffset(0.4)
    ratio.GetXaxis().SetTitleSize(0.12)
    ratio.GetXaxis().SetTitleOffset(0.85)
    Ratio.append(ratio)
    #ratio.SetAxisRange(50,150)

    ratio.Draw("EP")
    c[i].cd()

    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTJets,"t#bar{t} (aMC@NLO)","f")
    l[i].AddEntry(SingleTop,"Single top","f")
    l[i].AddEntry(WJets,"W+Jets","f")
    l[i].AddEntry(QCD,"QCD multijets","f")
    l[i].AddEntry(VV,"Diboson","f")
    l[i].AddEntry(Data_SingleMuonUL17,"Data UL 2017","lep")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("WSel/UL17/"+Hists_WSel[i]+"_aMCatNLO_UL17.png")
    c[i].Close()

0 recoJet_pt_nom_WSel ../SingleMuonAll_2017UL.root [20, 0, 1000] 41472.052 722.8 286629789811.7677
4 ../ST_s-channel_amcatnlo_pythia8_2017UL.root 3.549 70439807.475972
5 ../ST_t-channel_top_powheg_pythia8_2017UL.root 119.7 656310119.274
6 ../ST_t-channel_antitop_powheg_pythia8_2017UL.root 71.74 265124234.844
7 ../ST_tW_top_powheg_pythia8_2017UL.root 32.45 331239851.9677
8 ../ST_tW_antitop_powheg_pythia8_2017UL.root 32.51 303960792.4356
9 ../QCD_Muen_Pt_170to300_pythia8_2017UL.root 7055.0 36027673.0
10 ../QCD_Muen_Pt_300to470_pythia8_2017UL.root 619.3 494796.07288
11 ../QCD_Muen_Pt_470to600_pythia8_2017UL.root 59.24 517383.713837
12 ../QCD_Muen_Pt_600to800_pythia8_2017UL.root 18.21 17318812.571925
13 ../QCD_Muen_Pt_800to1000_pythia8_2017UL.root 25.97 16962615.0
P_{t}/GeV
1 recoJet_mass_nom_WSel ../SingleMuonAll_2017UL.root [11, 50, 160] 41472.052 722.8 286629789811.7677
4 ../ST_s-channel_amcatnlo_pythia8_2017UL.root 3.549 70439807.475972
5 ../ST_t-channel_top_powheg_pythia8_2017UL.root 

11 recoJet_tau_0p5_4_nom_WSel ../SingleMuonAll_2017UL.root [20, 0, 0.4] 41472.052 722.8 286629789811.7677
4 ../ST_s-channel_amcatnlo_pythia8_2017UL.root 3.549 70439807.475972
5 ../ST_t-channel_top_powheg_pythia8_2017UL.root 119.7 656310119.274
6 ../ST_t-channel_antitop_powheg_pythia8_2017UL.root 71.74 265124234.844
7 ../ST_tW_top_powheg_pythia8_2017UL.root 32.45 331239851.9677
8 ../ST_tW_antitop_powheg_pythia8_2017UL.root 32.51 303960792.4356
9 ../QCD_Muen_Pt_170to300_pythia8_2017UL.root 7055.0 36027673.0
10 ../QCD_Muen_Pt_300to470_pythia8_2017UL.root 619.3 494796.07288
11 ../QCD_Muen_Pt_470to600_pythia8_2017UL.root 59.24 517383.713837
12 ../QCD_Muen_Pt_600to800_pythia8_2017UL.root 18.21 17318812.571925
13 ../QCD_Muen_Pt_800to1000_pythia8_2017UL.root 25.97 16962615.0
#tau_{4}^{(0.5)}
12 recoJet_tau_1_4_nom_WSel ../SingleMuonAll_2017UL.root [20, 0, 0.2] 41472.052 722.8 286629789811.7677
4 ../ST_s-channel_amcatnlo_pythia8_2017UL.root 3.549 70439807.475972
5 ../ST_t-channel_top_powheg_pyt

23 AK8jets_mass_WSel ../SingleMuonAll_2017UL.root [15, 0, 150] 41472.052 722.8 286629789811.7677
4 ../ST_s-channel_amcatnlo_pythia8_2017UL.root 3.549 70439807.475972
5 ../ST_t-channel_top_powheg_pythia8_2017UL.root 119.7 656310119.274
6 ../ST_t-channel_antitop_powheg_pythia8_2017UL.root 71.74 265124234.844
7 ../ST_tW_top_powheg_pythia8_2017UL.root 32.45 331239851.9677
8 ../ST_tW_antitop_powheg_pythia8_2017UL.root 32.51 303960792.4356
9 ../QCD_Muen_Pt_170to300_pythia8_2017UL.root 7055.0 36027673.0
10 ../QCD_Muen_Pt_300to470_pythia8_2017UL.root 619.3 494796.07288
11 ../QCD_Muen_Pt_470to600_pythia8_2017UL.root 59.24 517383.713837
12 ../QCD_Muen_Pt_600to800_pythia8_2017UL.root 18.21 17318812.571925
13 ../QCD_Muen_Pt_800to1000_pythia8_2017UL.root 25.97 16962615.0
Mass of AK8Jets/GeV
24 nAK4jets_WSel ../SingleMuonAll_2017UL.root [8, 0, 8] 41472.052 722.8 286629789811.7677
4 ../ST_s-channel_amcatnlo_pythia8_2017UL.root 3.549 70439807.475972
5 ../ST_t-channel_top_powheg_pythia8_2017UL.root 119

Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL17/recoJet_pt_nom_WSel_aMCatNLO_UL17.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL17/recoJet_mass_nom_WSel_aMCatNLO_UL17.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL17/recoJet_tau_0p5_1_nom_WSel_aMCatNLO_UL17.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL17/recoJet_tau_1_1_nom_WSel_aMCatNLO_UL17.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL17/recoJet_tau_2_1_nom_WSel_aMCatNLO_UL17.pn

### Resolution plots


In [16]:
Hists_WSel=np.array([
    
    'resolJet_tau_0p5_1_WSel', 'resolJet_tau_1_1_WSel', 'resolJet_tau_2_1_WSel',
    'resolJet_tau_0p5_2_WSel', 'resolJet_tau_1_2_WSel', 'resolJet_tau_2_2_WSel',
    'resolJet_tau_0p5_3_WSel', 'resolJet_tau_1_3_WSel', 'resolJet_tau_2_3_WSel',
    'resolJet_tau_0p5_4_WSel', 'resolJet_tau_1_4_WSel', 'resolJet_tau_2_4_WSel',
    'resolJet_tau21_WSel', 'resolJet_tau32_WSel',
])
#Objects_WSel=np.concatenate((Objects1_WSel,Objects3_WSel),axis=0)

Histnames_WSel=np.array([
    ['Events','#resol tau_{1}^{(0.5)}','resol #tau_{1}^{(0.5)}'],['Events','resol #tau_{1}^{(1)}','resol #tau_{1}^{(1)}'],['Events','resol #tau_{1}^{(2)}','resol #tau_{1}^{(2)}'],
    ['Events','#resol tau_{2}^{(0.5)}','resol #tau_{2}^{(0.5)}'],['Events','resol #tau_{2}^{(1)}','resol #tau_{2}^{(1)}'],['Events','resol #tau_{2}^{(2)}','resol #tau_{2}^{(2)}'],
    ['Events','#resol tau_{3}^{(0.5)}','resol #tau_{3}^{(0.5)}'],['Events','resol #tau_{3}^{(1)}','resol #tau_{3}^{(1)}'],['Events','resol #tau_{3}^{(2)}','resol #tau_{3}^{(2)}'],
    ['Events','#resol tau_{4}^{(0.5)}','resol #tau_{4}^{(0.5)}'],['Events','resol #tau_{4}^{(1)}','resol #tau_{4}^{(1)}'],['Events','resol #tau_{4}^{(2)}','resol #tau_{4}^{(2)}'],
    ['Events','resol #tau_{2,1}^{(1)}','resol #tau_{2,1}^{(1)}'],['Events','resol #tau_{3,2}^{(1)}','resol #tau_{3,2}^{(1)}'],
    ])
Range_WSel=[
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.]
    
    
]
print (len(Hists_WSel),len(Histnames_WSel),len(Range_WSel))

14 14 14


In [17]:
print (Histnames_WSel[0][0]
      )

Events


In [18]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.12

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [19]:
c = np.zeros(len(Range_WSel), ROOT.TCanvas)
l = np.zeros(len(Range_WSel), ROOT.TLegend)

H_ref = 720; 
W_ref = 1024; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.97
y1_l = 0.85

dx_l = 0.3
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_WSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_WSel)):
    
    print (i, Hists_WSel[i], fpathData, Range_WSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],0,0,W,H)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    c[i].SetLeftMargin( L/W )
    c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], rebin=False))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetLineWidth(1)
    TTSemileptonic.SetMarkerSize(1)
    TTSemileptonic.SetMarkerStyle(26)
    TTSemileptonic.SetMarkerColor(ROOT.kGreen+3)
    TTSemileptonic.SetFillStyle(0)
    
    TTJets = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[0], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[0], nevt=nevts[0], rebin=False))
    TTJets.SetLineColor(ROOT.kRed+5)
    TTJets.SetLineWidth(1)
    TTJets.SetMarkerSize(1)
    TTJets.SetMarkerStyle(26)
    TTJets.SetMarkerColor(ROOT.kRed+5)
    TTJets.SetFillStyle(0)
    
    
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    TTSemileptonic.GetYaxis().SetTitle(Histnames_WSel[i][0])
    TTSemileptonic.GetXaxis().SetTitle(Histnames_WSel[i][2]) 
    print(Histnames_WSel[i][2])
    #TTSemileptonic.GetYaxis().SetRangeUser(-0.5, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    TTSemileptonic.Rebin(2)
    TTJets.Rebin(2)
    TTSemileptonic.Draw("ep")
    TTJets.Draw("ep samee")
    
    
    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} semileptonic (Powheg)","p")
    l[i].AddEntry(TTJets,"t#bar{t} inclusive (aMC@NLO)","p")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("WSel/UL17/"+Hists_WSel[i]+".png")
    c[i].Close()

0 resolJet_tau_0p5_1_WSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 41472.052 365.34 32392787477.907993
resol #tau_{1}^{(0.5)}
1 resolJet_tau_1_1_WSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 41472.052 365.34 32392787477.907993
resol #tau_{1}^{(1)}
2 resolJet_tau_2_1_WSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 41472.052 365.34 32392787477.907993
resol #tau_{1}^{(2)}
3 resolJet_tau_0p5_2_WSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 41472.052 365.34 32392787477.907993
resol #tau_{2}^{(0.5)}
4 resolJet_tau_1_2_WSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 41472.052 365.34 32392787477.907993
resol #tau_{2}^{(1)}
5 resolJet_tau_2_2_WSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 41472.052 365.34 32392787477.907993
resol #tau_{2}^{(2)}
6 resolJet_tau_0p5_3_WSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 41472.052 365.34 32392787477.907993
resol #tau_{3}^{(0.5)}
7 resolJet_tau_1_3_WSel ../SingleMuonAll_2017UL.root [50, 0.0, 2.0] 41472.052 365.34 32392787477.907993
resol #ta

Info in <TCanvas::Print>: png file WSel/UL17/resolJet_tau_0p5_1_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL17/resolJet_tau_1_1_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL17/resolJet_tau_2_1_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL17/resolJet_tau_0p5_2_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL17/resolJet_tau_1_2_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL17/resolJet_tau_2_2_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL17/resolJet_tau_0p5_3_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL17/resolJet_tau_1_3_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL17/resolJet_tau_2_3_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL17/resolJet_tau_0p5_4_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL17/resolJet_tau_1_4_WSel.png has been created
Info in <TCanvas::Print>

### UL 2018 

In [20]:
dict_MCscaling_UL18 = {
    'TTJets': ["722.8", 297902725633.38257,
               '../TTJets_amcatnloFXFX-pythia8_2018UL.root',
              ],
    'TTToSemiLeptonic': ["365.34", 31378075292.893986,
                         '../TTToSemileptonic_powheg_pythia8_2018UL.root',
                        ],
    'TTTo2L2Nu': ["88.29 ", 5063754431.9676, 
                  '../TTTo2L2Nu_powheg_pythia8_2018UL.root',
                 ],
    'WJetsToLNu': ["5.368e+04", 10484986870276.357422, 
                   '../WJetsToLNu_madgraphMLM_pythia8_2018UL.root',
                  ],
    'ST_s-channel': ["3.549e+00", 64014403.924644, 
                     '../ST_s-channel_amcatnlo_pythia8_2018UL.root',
                    ],
    'ST_t-channel_top': ["1.197e+02", 683747124.990000, 
                         '../ST_t-channel_top_powheg_pythia8_2018UL.root'
                        ],
    'ST_t-channel_antitop': ["7.174e+01", 271219283.652000, 
                            '../ST_t-channel_antitop_powheg_pythia8_2018UL.root'
                            ],
    'ST_tW_top': ["3.245e+01", 328321574.253000, 
                  '../ST_tW_top_powheg_pythia8_2018UL.root',
                 ],
    'ST_tW_antitop': ["3.251e+01", 296655487.557600, 
                     '../ST_tW_antitop_powheg_pythia8_2018UL.root',
                     ],
    #'QCD_Pt_170to300': ["7055.0", 36027673.000000, 
    #                    '../QCD_Muen_Pt_170to300_pythia8_2018UL.root',
    #                   ],
    #'QCD_Pt_300to470': ["619.3", 494796.072880, 
    #                    '../QCD_Muen_Pt_300to470_pythia8_2018UL.root',   
    #                   ],
    #'QCD_Pt_470to600': ["59.24", 517383.713837, 
    #                    '../QCD_Muen_Pt_470to600_pythia8_2018UL.root',
    #                   ],
    #'QCD_Pt_600to800': ["18.21", 17318812.571925, 
    #                    '../QCD_Muen_Pt_600to800_pythia8_2018UL.root',
    #                   ],
    #'QCD_Pt_800to1000': ["2.597e+01", 39261600.000000, 
    #                     '../QCD_Pt_800to1000_pythia8_2018UL.root'    
    #                    ],
    #'QCD_Pt_1000toInf': ["1.078", 14642553.000000, 
    #                      '../QCD_Muen_Pt_1000toInf_pythia8_2018UL.root',
    #                     ],
    
    'WW': ["7.577e+01", 7959266.140092, 
          '../WW_pythia8_2018UL.root'  
          ],
    'WZ': ["1.21e+00", 3976000.000000, 
           '../WZ_pythia8_2018UL.root'    
          ],
    'ZZ': ["2.748e+00", 2000000.000000, 
           '../ZZ_pythia8_2018UL.root'
          ]
    }



In [21]:
lumi=59816.39#41472.052#4653.71+9289.53+4247.68+9312.83+11758.83
print (lumi)

samples = list(dict_MCscaling_UL18.keys())
dict_items = list(dict_MCscaling_UL18.items())
xsecs = np.array(list(float(dict_items[k][1][0]) for k in np.arange(0,len(dict_items)))) 
nevts = np.array(list(float(dict_items[k][1][1]) for k in np.arange(0,len(dict_items)))) 
fpathsMC = np.array(list((dict_items[k][1][2]) for k in np.arange(0,len(dict_items))))
fpathData = '../SingleMuonAll_2018UL.root'
for i in range(0,len(samples)):
    print (i, xsecs[i], samples[i], nevts[i], fpathsMC[i])

59816.39
0 722.8 TTJets 297902725633.38257 ../TTJets_amcatnloFXFX-pythia8_2018UL.root
1 365.34 TTToSemiLeptonic 31378075292.893986 ../TTToSemileptonic_powheg_pythia8_2018UL.root
2 88.29 TTTo2L2Nu 5063754431.9676 ../TTTo2L2Nu_powheg_pythia8_2018UL.root
3 53680.0 WJetsToLNu 10484986870276.357 ../WJetsToLNu_madgraphMLM_pythia8_2018UL.root
4 3.549 ST_s-channel 64014403.924644 ../ST_s-channel_amcatnlo_pythia8_2018UL.root
5 119.7 ST_t-channel_top 683747124.99 ../ST_t-channel_top_powheg_pythia8_2018UL.root
6 71.74 ST_t-channel_antitop 271219283.652 ../ST_t-channel_antitop_powheg_pythia8_2018UL.root
7 32.45 ST_tW_top 328321574.253 ../ST_tW_top_powheg_pythia8_2018UL.root
8 32.51 ST_tW_antitop 296655487.5576 ../ST_tW_antitop_powheg_pythia8_2018UL.root
9 75.77 WW 7959266.140092 ../WW_pythia8_2018UL.root
10 1.21 WZ 3976000.0 ../WZ_pythia8_2018UL.root
11 2.748 ZZ 2000000.0 ../ZZ_pythia8_2018UL.root


In [22]:
print (len((dict_MCscaling_UL18.keys())))

12


In [23]:
import array
from array import array

In [24]:
def Bin(Range):
    bins=np.array([x for x in np.linspace(num=Range[0]+1,start=Range[1],stop=Range[2])])
    return bins

def getRebinnedRescaledHist(hist_name, file_name, xrange, lumi, xsec, nevt, isMC=True, rebin=True):
    #if not(sample_name in file_name): 
    #    return "File either not available or needs to be renamed to be compatible with this (poorly,) hard-coded piece of software..."
    
    FILE = ROOT.TFile.Open(file_name,'read')
    tree = FILE.Get('jetObservables')
    hist = tree.Get(hist_name)
    hist.SetDirectory(0)
    
    ROOT.TH1.AddDirectory(ROOT.kFALSE);
    
    bins = array('d', np.array(Bin(xrange)))
    
    if rebin: newHist = ROOT.TH1F(hist.Rebin(len(bins)-1, "%s_rebin"%hist_name, bins))
    else: newHist = hist.Clone()
    #newHist.SetStats(ROOT.kFALSE)
    if isMC==True: newHist.Scale(lumi*xsec/nevt)
    
    return newHist

In [25]:
Hists_WSel=np.array([
    'recoJet_pt_nom_WSel', 'recoJet_mass_nom_WSel', #'recoJetTau21_nom_WSel', 'recoJetTau32_nom_WSel',
    'recoJet_tau_0p5_1_nom_WSel', 'recoJet_tau_1_1_nom_WSel', 'recoJet_tau_2_1_nom_WSel',
    'recoJet_tau_0p5_2_nom_WSel', 'recoJet_tau_1_2_nom_WSel', 'recoJet_tau_2_2_nom_WSel',
    'recoJet_tau_0p5_3_nom_WSel', 'recoJet_tau_1_3_nom_WSel', 'recoJet_tau_2_3_nom_WSel',
    'recoJet_tau_0p5_4_nom_WSel', 'recoJet_tau_1_4_nom_WSel', 'recoJet_tau_2_4_nom_WSel',
    'nPVs_WSel','nleps_WSel',
    'muons_pt_WSel','muons_eta_WSel','muons_phi_WSel',
    'nAK8jets_WSel','AK8jets_pt_WSel','AK8jets_eta_WSel','AK8jets_phi_WSel','AK8jets_mass_WSel',
    'nAK4jets_WSel','AK4jets_pt_WSel','AK4jets_eta_WSel','AK4jets_phi_WSel','AK4jets_mass_WSel',
    'METPt_WSel','HT_WSel'])
#Objects_WSel=np.concatenate((Objects1_WSel,Objects3_WSel),axis=0)

Histnames_WSel=np.array([
    ['Events/(50 GeV)','P_{t}','P_{t}/GeV'],['Events/(10 GeV)','Softdrop Mass','Softdrop Mass/GeV'],#['Events/0.05','#tau_{21}','#tau_{21}'],['Events/0.05','#tau_{32}','#tau_{32}'],
    ['Events/0.03','#tau_{1}^{(0.5)}','#tau_{1}^{(0.5)}'],['Events/0.02','#tau_{1}^{(1)}','#tau_{1}^{(1)}'],['Events/0.02','#tau_{1}^{(2)}','#tau_{1}^{(2)}'],
    ['Events/0.03','#tau_{2}^{(0.5)}','#tau_{2}^{(0.5)}'],['Events/0.02','#tau_{2}^{(1)}','#tau_{2}^{(1)}'],['Events/0.01','#tau_{2}^{(2)}','#tau_{2}^{(2)}'],
    ['Events/0.02','#tau_{3}^{(0.5)}','#tau_{3}^{(0.5)}'],['Events/0.02','#tau_{3}^{(1)}','#tau_{3}^{(1)}'],['Events/0.01','#tau_{3}^{(2)}','#tau_{3}^{(2)}'],
    ['Events/0.02','#tau_{4}^{(0.5)}','#tau_{4}^{(0.5)}'],['Events/0.01','#tau_{4}^{(1)}','#tau_{4}^{(1)}'],['Events/0.01','#tau_{4}^{(2)}','#tau_{4}^{(2)}'],
    ['Events/2','Number of PV','Number of PV'],['Events','Number of Leptons','Number of Leptons'],
    ['Events/(20 GeV)','P_{t} of Muons','P_{t} of Muons/GeV'],['Events/0.4','#eta of Muons','#eta of Muons'],['Events/(#pi/10)','#phi of Muons','#phi of Muons'],
    ['Events','Number of AK8Jets','Number of AK8Jets'],['Events/(60 GeV)','P_{t} of AK8Jets','P_{t} of AK8Jets/GeV'],['Events/0.4','#eta of AK8Jets','#eta of AK8Jets'],['Events/(#pi/10)','#phi of AK8Jets','#phi of AK8Jets'],['Events/(10 GeV)','Mass of AK8Jets','Mass of AK8Jets/GeV'],
    ['Events','Number of AK4Jets','Number of AK4Jets'],['Events/(40 GeV)','P_{t} of AK4Jets','P_{t} of AK4Jets/GeV'],['Events/0.4','#eta of AK4Jets','#eta of AK4Jets'],['Events/(#pi/10)','#phi of AK4Jets','#phi of AK4Jets'],['Events/(20 GeV)','Mass of AK4Jets','Mass of AK4Jets/GeV'],
    ['Events/(20 GeV)','MET P_{t}','MET P_{t}/GeV'],['Events/(100 GeV)','HT','HT/GeV']])
Range_WSel=[
    [20,0,1000], [11,50,160],#[20,0,1],[20,0,1],
    [30,0,0.9],[35,0,0.7],[20,0,0.4],
    [20,0,0.6],[20,0,0.4],[20,0,0.2],
    [25,0,0.5],[20,0,0.4],[20,0,0.1],
    [20,0,0.4],[20,0,0.2],[20,0,0.1],  
    [35,0,70],[4,0,4],
    [20,0,400],[20,-4,4],[20,-3.14259, 3.14159],
    [8,0,8],[20,0,1200],[20,-4,4],[20,-3.14259, 3.14159],[15,0,150],
    [8,0,8],[20,0,800],[20,-4,4],[20,-3.14259, 3.14159],[20,0,400],
    [30,0,600],[20,0,2000]
]
print (len(Hists_WSel),len(Histnames_WSel),len(Range_WSel))

31 31 31


In [26]:
print (Histnames_WSel[0][0]
      )

Events/(50 GeV)


In [27]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.12

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [28]:
c = np.zeros(len(Range_WSel), ROOT.TCanvas)
l = np.zeros(len(Range_WSel), ROOT.TLegend)

H_ref = 720; 
W_ref = 1024; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.97
y1_l = 0.85

dx_l = 0.35
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_WSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_WSel)):
    
    print (i, Hists_WSel[i], fpathData, Range_WSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],0,0,W,H)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    c[i].SetLeftMargin( L/W )
    c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 0.95, -1,0,0);
    pad1.SetBottomMargin(0.); #// Upper and lower plot are joined
    #pad1.SetGridx();         #// Vertical grid
    pad1.SetBorderMode(0)
    pad1.SetBorderSize(0)
    pad1.SetFrameLineWidth(0)
    pad1.SetFrameBorderMode(0)
    pad1.Draw()           #// Draw the upper pad: pad1
    pad1.cd()
    Pad1.append(pad1)
    
    
    

    MC = ROOT.THStack("MC", Hists_WSel[i])
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1]))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetFillColor(ROOT.kGreen+3)
    
    TTDileptonic = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[2], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[2], nevt=nevts[2]))
    TTDileptonic.SetLineColor(ROOT.kRed-9)
    TTDileptonic.SetFillColor(ROOT.kRed-9)
    
    WJets = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[3], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[3], nevt=nevts[3]))
    WJets.SetLineColor(ROOT.kCyan-6)
    WJets.SetFillColor(ROOT.kCyan-6)
    
    SingleTop = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(4,9):
        #print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                               file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                               lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        SingleTop.Add(hist)
        
    SingleTop.SetLineColor(ROOT.kAzure+8)
    SingleTop.SetFillColor(ROOT.kAzure+8)
    
    #QCD = ROOT.TH1F("",";;",*Range_WSel[i])
    #for j in range(9,14):
    #    #print (j,fpathsMC[j],xsecs[j],nevts[j])
    #    hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
    #                                     file_name=fpathsMC[j], xrange=Range_WSel[i], 
    #                                     lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
    #    QCD.Add(hist)
    #QCD.SetLineColor(ROOT.kPink-1)
    #QCD.SetFillColor(ROOT.kPink-1)
    
    VV = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(9,12):
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                        file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                        lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        VV.Add(hist)
    VV.SetLineColor(ROOT.kGray+2)
    VV.SetFillColor(ROOT.kGray+2)
    
    #VV.GetYaxis().SetRangeUser(-0.5, 1.25*TTSemileptonic.GetMaximum())
    MC.Add(VV)
    MC.Add(SingleTop)
    #MC.Add(QCD)
    MC.Add(WJets)
    MC.Add(TTDileptonic)
    MC.Add(TTSemileptonic)
    dummy = MC.GetHists().At(0)
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    dummy.Draw()  
    dummy.GetYaxis().SetTitle(Histnames_WSel[i][0])
    dummy.GetXaxis().SetTitle(Histnames_WSel[i][2]) 
    print(Histnames_WSel[i][2])
    dummy.GetYaxis().SetRangeUser(-0.5, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    MC.Draw("hist e")
    
    c[i].Update()
    MC.Draw("histsame e")
    Data_SingleMuonUL18 = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathData, xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], isMC=False))
    Data_SingleMuonUL18.SetMarkerSize(1.5)
    Data_SingleMuonUL18.SetMarkerColor(ROOT.kBlack)
    Data_SingleMuonUL18.SetMarkerStyle(20)
    Data_SingleMuonUL18.SetLineColor(ROOT.kBlack)
    Data_SingleMuonUL18.Draw("epsamex0")
    
    
    CMS_lumi.CMS_lumi(c[i], iPeriod, iPos)
    c[i].cd()
    c[i].Update()
    c[i].RedrawAxis()
    frame = c[i].GetFrame()
    frame.Draw()

    pad2 = ROOT.TPad("pad2","pad2",0,0.0,1,0.3, -1,0,0)
    pad2.SetTopMargin(0)
    pad2.SetGridx()
    pad2.SetGridy()
    pad2.Draw()
    pad2.cd()
    pad2.SetBottomMargin(0.3)
    Pad2.append(pad2)

    ratio = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    ratio.GetXaxis().SetTitle(Histnames_WSel[i][2])
    ratio.Add(Data_SingleMuonUL18)
    ratio.GetYaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetXaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetYaxis().SetRangeUser(0.3,1.7)
    
    mc_rt = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    mc_rt.Add(TTSemileptonic)
    mc_rt.Add(TTDileptonic)
    mc_rt.Add(SingleTop)
    #mc_rt.Add(QCD)
    mc_rt.Add(VV)
    mc_rt.Add(WJets)
    
    ratio.GetYaxis().SetTitle("#frac{Data}{MC}")
    ratio.Sumw2()
    ratio.SetStats(0)
    ratio.Divide(mc_rt)
    ratio.GetYaxis().SetLabelSize(0.09)
    ratio.GetXaxis().SetLabelSize(0.09)
    ratio.SetMarkerColor(ROOT.kBlack)
    ratio.SetMarkerStyle(20)
    ratio.SetLineColor(ROOT.kBlack)
    ratio.GetYaxis().SetTitleSize(0.12)
    ratio.GetYaxis().SetTitleOffset(0.4)
    ratio.GetXaxis().SetTitleSize(0.12)
    ratio.GetXaxis().SetTitleOffset(0.85)
    Ratio.append(ratio)
    #ratio.SetAxisRange(50,150)

    ratio.Draw("EP")
    c[i].cd()

    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} semileptonic (Powheg)","f")
    l[i].AddEntry(TTDileptonic,"t#bar{t} dileptonic","f")
    l[i].AddEntry(SingleTop,"Single top","f")
    l[i].AddEntry(WJets,"W+Jets","f")
    #l[i].AddEntry(QCD,"QCD multijets","f")
    l[i].AddEntry(VV,"Diboson","f")
    l[i].AddEntry(Data_SingleMuonUL18,"Data UL 2018","lep")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("WSel/UL18/"+Hists_WSel[i]+"_powheg_p8_UL18.png")
    c[i].Close()

0 recoJet_pt_nom_WSel ../SingleMuonAll_2018UL.root [20, 0, 1000] 59816.39 365.34 31378075292.893986
P_{t}/GeV
1 recoJet_mass_nom_WSel ../SingleMuonAll_2018UL.root [11, 50, 160] 59816.39 365.34 31378075292.893986
Softdrop Mass/GeV
2 recoJet_tau_0p5_1_nom_WSel ../SingleMuonAll_2018UL.root [30, 0, 0.9] 59816.39 365.34 31378075292.893986
#tau_{1}^{(0.5)}
3 recoJet_tau_1_1_nom_WSel ../SingleMuonAll_2018UL.root [35, 0, 0.7] 59816.39 365.34 31378075292.893986
#tau_{1}^{(1)}
4 recoJet_tau_2_1_nom_WSel ../SingleMuonAll_2018UL.root [20, 0, 0.4] 59816.39 365.34 31378075292.893986
#tau_{1}^{(2)}
5 recoJet_tau_0p5_2_nom_WSel ../SingleMuonAll_2018UL.root [20, 0, 0.6] 59816.39 365.34 31378075292.893986
#tau_{2}^{(0.5)}
6 recoJet_tau_1_2_nom_WSel ../SingleMuonAll_2018UL.root [20, 0, 0.4] 59816.39 365.34 31378075292.893986
#tau_{2}^{(1)}
7 recoJet_tau_2_2_nom_WSel ../SingleMuonAll_2018UL.root [20, 0, 0.2] 59816.39 365.34 31378075292.893986
#tau_{2}^{(2)}
8 recoJet_tau_0p5_3_nom_WSel ../SingleMuonAll_20

Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL18/recoJet_pt_nom_WSel_powheg_p8_UL18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL18/recoJet_mass_nom_WSel_powheg_p8_UL18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL18/recoJet_tau_0p5_1_nom_WSel_powheg_p8_UL18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL18/recoJet_tau_1_1_nom_WSel_powheg_p8_UL18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL18/recoJet_tau_2_1_nom_WSel_powheg_p8_UL

In [29]:
c = np.zeros(len(Range_WSel), ROOT.TCanvas)
l = np.zeros(len(Range_WSel), ROOT.TLegend)

H_ref = 720; 
W_ref = 1024; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.97
y1_l = 0.85

dx_l = 0.35
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_WSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_WSel)):
    
    print (i, Hists_WSel[i], fpathData, Range_WSel[i], lumi, xsecs[0], nevts[0])

    c[i] = ROOT.TCanvas(CName[i],CName[i],0,0,W,H)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    c[i].SetLeftMargin( L/W )
    c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 0.95, -1,0,0);
    pad1.SetBottomMargin(0.); #// Upper and lower plot are joined
    #pad1.SetGridx();         #// Vertical grid
    pad1.SetBorderMode(0)
    pad1.SetBorderSize(0)
    pad1.SetFrameLineWidth(0)
    pad1.SetFrameBorderMode(0)
    pad1.Draw()           #// Draw the upper pad: pad1
    pad1.cd()
    Pad1.append(pad1)
    
    
    

    MC = ROOT.THStack("MC", Hists_WSel[i])
    
    TTJets = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[0], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[0], nevt=nevts[0]))
    TTJets.SetLineColor(ROOT.kGreen+3)
    TTJets.SetFillColor(ROOT.kGreen+3)
    
    
    WJets = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[3], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[3], nevt=nevts[3]))
    WJets.SetLineColor(ROOT.kCyan-6)
    WJets.SetFillColor(ROOT.kCyan-6)
    
    SingleTop = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(4,9):
        print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                               file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                               lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        SingleTop.Add(hist)
        
    SingleTop.SetLineColor(ROOT.kAzure+8)
    SingleTop.SetFillColor(ROOT.kAzure+8)
    
    '''
    QCD = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(9,14):
        print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                         file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                         lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        QCD.Add(hist)
    QCD.SetLineColor(ROOT.kPink-1)
    QCD.SetFillColor(ROOT.kPink-1)
    '''
    VV = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(9,12):
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                        file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                        lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        VV.Add(hist)
    VV.SetLineColor(ROOT.kGray+2)
    VV.SetFillColor(ROOT.kGray+2)
    
    #VV.GetYaxis().SetRangeUser(-0.5, 1.25*TTJets.GetMaximum())
    MC.Add(VV)
    MC.Add(SingleTop)
    #MC.Add(QCD)
    MC.Add(WJets)
    MC.Add(TTJets)
    dummy = MC.GetHists().At(0)
    #dummy.SetAxisRange(0., TTJets.GetMaximum()*1.5)
    dummy.Draw()  
    dummy.GetYaxis().SetTitle(Histnames_WSel[i][0])
    dummy.GetXaxis().SetTitle(Histnames_WSel[i][2]) 
    print(Histnames_WSel[i][2])
    dummy.GetYaxis().SetRangeUser(-0.5, 2.5*TTJets.GetMaximum())
    c[i].Update()
    MC.Draw("hist e")
    
    c[i].Update()
    MC.Draw("histsame e")
    Data_SingleMuonUL18 = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathData, xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[0], nevt=nevts[0], isMC=False))
    Data_SingleMuonUL18.SetMarkerSize(1.5)
    Data_SingleMuonUL18.SetMarkerColor(ROOT.kBlack)
    Data_SingleMuonUL18.SetMarkerStyle(20)
    Data_SingleMuonUL18.SetLineColor(ROOT.kBlack)
    Data_SingleMuonUL18.Draw("epsamex0")
    
    
    CMS_lumi.CMS_lumi(c[i], iPeriod, iPos)
    c[i].cd()
    c[i].Update()
    c[i].RedrawAxis()
    frame = c[i].GetFrame()
    frame.Draw()

    pad2 = ROOT.TPad("pad2","pad2",0,0.0,1,0.3, -1,0,0)
    pad2.SetTopMargin(0)
    pad2.SetGridx()
    pad2.SetGridy()
    pad2.Draw()
    pad2.cd()
    pad2.SetBottomMargin(0.3)
    Pad2.append(pad2)

    ratio = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    ratio.GetXaxis().SetTitle(Histnames_WSel[i][2])
    ratio.Add(Data_SingleMuonUL18)
    ratio.GetYaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetXaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetYaxis().SetRangeUser(0.3,1.7)
    
    mc_rt = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    mc_rt.Add(TTJets)
    mc_rt.Add(SingleTop)
    #mc_rt.Add(QCD)
    mc_rt.Add(VV)
    mc_rt.Add(WJets)
    
    ratio.GetYaxis().SetTitle("#frac{Data}{MC}")
    ratio.Sumw2()
    ratio.SetStats(0)
    ratio.Divide(mc_rt)
    ratio.GetYaxis().SetLabelSize(0.09)
    ratio.GetXaxis().SetLabelSize(0.09)
    ratio.SetMarkerColor(ROOT.kBlack)
    ratio.SetMarkerStyle(20)
    ratio.SetLineColor(ROOT.kBlack)
    ratio.GetYaxis().SetTitleSize(0.12)
    ratio.GetYaxis().SetTitleOffset(0.4)
    ratio.GetXaxis().SetTitleSize(0.12)
    ratio.GetXaxis().SetTitleOffset(0.85)
    Ratio.append(ratio)
    #ratio.SetAxisRange(50,150)

    ratio.Draw("EP")
    c[i].cd()

    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTJets,"t#bar{t} (aMC@NLO)","f")
    l[i].AddEntry(SingleTop,"Single top","f")
    l[i].AddEntry(WJets,"W+Jets","f")
    #l[i].AddEntry(QCD,"QCD multijets","f")
    l[i].AddEntry(VV,"Diboson","f")
    l[i].AddEntry(Data_SingleMuonUL18,"Data UL 2018","lep")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("WSel/UL18/"+Hists_WSel[i]+"_aMCatNLO_UL18.png")
    c[i].Close()

0 recoJet_pt_nom_WSel ../SingleMuonAll_2018UL.root [20, 0, 1000] 59816.39 722.8 297902725633.38257
4 ../ST_s-channel_amcatnlo_pythia8_2018UL.root 3.549 64014403.924644
5 ../ST_t-channel_top_powheg_pythia8_2018UL.root 119.7 683747124.99
6 ../ST_t-channel_antitop_powheg_pythia8_2018UL.root 71.74 271219283.652
7 ../ST_tW_top_powheg_pythia8_2018UL.root 32.45 328321574.253
8 ../ST_tW_antitop_powheg_pythia8_2018UL.root 32.51 296655487.5576
P_{t}/GeV
1 recoJet_mass_nom_WSel ../SingleMuonAll_2018UL.root [11, 50, 160] 59816.39 722.8 297902725633.38257
4 ../ST_s-channel_amcatnlo_pythia8_2018UL.root 3.549 64014403.924644
5 ../ST_t-channel_top_powheg_pythia8_2018UL.root 119.7 683747124.99
6 ../ST_t-channel_antitop_powheg_pythia8_2018UL.root 71.74 271219283.652
7 ../ST_tW_top_powheg_pythia8_2018UL.root 32.45 328321574.253
8 ../ST_tW_antitop_powheg_pythia8_2018UL.root 32.51 296655487.5576
Softdrop Mass/GeV
2 recoJet_tau_0p5_1_nom_WSel ../SingleMuonAll_2018UL.root [30, 0, 0.9] 59816.39 722.8 29790272

8 ../ST_tW_antitop_powheg_pythia8_2018UL.root 32.51 296655487.5576
Number of AK8Jets
20 AK8jets_pt_WSel ../SingleMuonAll_2018UL.root [20, 0, 1200] 59816.39 722.8 297902725633.38257
4 ../ST_s-channel_amcatnlo_pythia8_2018UL.root 3.549 64014403.924644
5 ../ST_t-channel_top_powheg_pythia8_2018UL.root 119.7 683747124.99
6 ../ST_t-channel_antitop_powheg_pythia8_2018UL.root 71.74 271219283.652
7 ../ST_tW_top_powheg_pythia8_2018UL.root 32.45 328321574.253
8 ../ST_tW_antitop_powheg_pythia8_2018UL.root 32.51 296655487.5576
P_{t} of AK8Jets/GeV
21 AK8jets_eta_WSel ../SingleMuonAll_2018UL.root [20, -4, 4] 59816.39 722.8 297902725633.38257
4 ../ST_s-channel_amcatnlo_pythia8_2018UL.root 3.549 64014403.924644
5 ../ST_t-channel_top_powheg_pythia8_2018UL.root 119.7 683747124.99
6 ../ST_t-channel_antitop_powheg_pythia8_2018UL.root 71.74 271219283.652
7 ../ST_tW_top_powheg_pythia8_2018UL.root 32.45 328321574.253
8 ../ST_tW_antitop_powheg_pythia8_2018UL.root 32.51 296655487.5576
#eta of AK8Jets
22 AK8jet

Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL18/recoJet_pt_nom_WSel_aMCatNLO_UL18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL18/recoJet_mass_nom_WSel_aMCatNLO_UL18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL18/recoJet_tau_0p5_1_nom_WSel_aMCatNLO_UL18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL18/recoJet_tau_1_1_nom_WSel_aMCatNLO_UL18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/UL18/recoJet_tau_2_1_nom_WSel_aMCatNLO_UL18.pn

### Resolution plots


In [30]:
Hists_WSel=np.array([
    
    'resolJet_tau_0p5_1_WSel', 'resolJet_tau_1_1_WSel', 'resolJet_tau_2_1_WSel',
    'resolJet_tau_0p5_2_WSel', 'resolJet_tau_1_2_WSel', 'resolJet_tau_2_2_WSel',
    'resolJet_tau_0p5_3_WSel', 'resolJet_tau_1_3_WSel', 'resolJet_tau_2_3_WSel',
    'resolJet_tau_0p5_4_WSel', 'resolJet_tau_1_4_WSel', 'resolJet_tau_2_4_WSel',
    'resolJet_tau21_WSel', 'resolJet_tau32_WSel',
])
#Objects_WSel=np.concatenate((Objects1_WSel,Objects3_WSel),axis=0)

Histnames_WSel=np.array([
    ['Events','#resol tau_{1}^{(0.5)}','resol #tau_{1}^{(0.5)}'],['Events','resol #tau_{1}^{(1)}','resol #tau_{1}^{(1)}'],['Events','resol #tau_{1}^{(2)}','resol #tau_{1}^{(2)}'],
    ['Events','#resol tau_{2}^{(0.5)}','resol #tau_{2}^{(0.5)}'],['Events','resol #tau_{2}^{(1)}','resol #tau_{2}^{(1)}'],['Events','resol #tau_{2}^{(2)}','resol #tau_{2}^{(2)}'],
    ['Events','#resol tau_{3}^{(0.5)}','resol #tau_{3}^{(0.5)}'],['Events','resol #tau_{3}^{(1)}','resol #tau_{3}^{(1)}'],['Events','resol #tau_{3}^{(2)}','resol #tau_{3}^{(2)}'],
    ['Events','#resol tau_{4}^{(0.5)}','resol #tau_{4}^{(0.5)}'],['Events','resol #tau_{4}^{(1)}','resol #tau_{4}^{(1)}'],['Events','resol #tau_{4}^{(2)}','resol #tau_{4}^{(2)}'],
    ['Events','resol #tau_{2,1}^{(1)}','resol #tau_{2,1}^{(1)}'],['Events','resol #tau_{3,2}^{(1)}','resol #tau_{3,2}^{(1)}'],
    ])
Range_WSel=[
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.]
    
    
]
print (len(Hists_WSel),len(Histnames_WSel),len(Range_WSel))

14 14 14


In [31]:
print (Histnames_WSel[0][0]
      )

Events


In [32]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.12

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [33]:
c = np.zeros(len(Range_WSel), ROOT.TCanvas)
l = np.zeros(len(Range_WSel), ROOT.TLegend)

H_ref = 720; 
W_ref = 1024; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.97
y1_l = 0.85

dx_l = 0.3
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_WSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_WSel)):
    
    print (i, Hists_WSel[i], fpathData, Range_WSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],0,0,W,H)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    c[i].SetLeftMargin( L/W )
    c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], rebin=False))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetLineWidth(1)
    TTSemileptonic.SetMarkerSize(1)
    TTSemileptonic.SetMarkerStyle(26)
    TTSemileptonic.SetMarkerColor(ROOT.kGreen+3)
    TTSemileptonic.SetFillStyle(0)
    
    TTJets = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[0], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[0], nevt=nevts[0], rebin=False))
    TTJets.SetLineColor(ROOT.kRed+5)
    TTJets.SetLineWidth(1)
    TTJets.SetMarkerSize(1)
    TTJets.SetMarkerStyle(26)
    TTJets.SetMarkerColor(ROOT.kRed+5)
    TTJets.SetFillStyle(0)
    
    
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    TTSemileptonic.GetYaxis().SetTitle(Histnames_WSel[i][0])
    TTSemileptonic.GetXaxis().SetTitle(Histnames_WSel[i][2]) 
    print(Histnames_WSel[i][2])
    #TTSemileptonic.GetYaxis().SetRangeUser(-0.5, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    TTSemileptonic.Rebin(2)
    TTJets.Rebin(2)
    TTSemileptonic.Draw("ep")
    TTJets.Draw("ep samee")
    
    
    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} semileptonic (Powheg)","p")
    l[i].AddEntry(TTJets,"t#bar{t} inclusive (aMC@NLO)","p")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("WSel/UL18/"+Hists_WSel[i]+".png")
    c[i].Close()

0 resolJet_tau_0p5_1_WSel ../SingleMuonAll_2018UL.root [50, 0.0, 2.0] 59816.39 365.34 31378075292.893986
resol #tau_{1}^{(0.5)}
1 resolJet_tau_1_1_WSel ../SingleMuonAll_2018UL.root [50, 0.0, 2.0] 59816.39 365.34 31378075292.893986
resol #tau_{1}^{(1)}
2 resolJet_tau_2_1_WSel ../SingleMuonAll_2018UL.root [50, 0.0, 2.0] 59816.39 365.34 31378075292.893986
resol #tau_{1}^{(2)}
3 resolJet_tau_0p5_2_WSel ../SingleMuonAll_2018UL.root [50, 0.0, 2.0] 59816.39 365.34 31378075292.893986
resol #tau_{2}^{(0.5)}
4 resolJet_tau_1_2_WSel ../SingleMuonAll_2018UL.root [50, 0.0, 2.0] 59816.39 365.34 31378075292.893986
resol #tau_{2}^{(1)}
5 resolJet_tau_2_2_WSel ../SingleMuonAll_2018UL.root [50, 0.0, 2.0] 59816.39 365.34 31378075292.893986
resol #tau_{2}^{(2)}
6 resolJet_tau_0p5_3_WSel ../SingleMuonAll_2018UL.root [50, 0.0, 2.0] 59816.39 365.34 31378075292.893986
resol #tau_{3}^{(0.5)}
7 resolJet_tau_1_3_WSel ../SingleMuonAll_2018UL.root [50, 0.0, 2.0] 59816.39 365.34 31378075292.893986
resol #tau_{3}^{(

Info in <TCanvas::Print>: png file WSel/UL18/resolJet_tau_0p5_1_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL18/resolJet_tau_1_1_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL18/resolJet_tau_2_1_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL18/resolJet_tau_0p5_2_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL18/resolJet_tau_1_2_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL18/resolJet_tau_2_2_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL18/resolJet_tau_0p5_3_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL18/resolJet_tau_1_3_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL18/resolJet_tau_2_3_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL18/resolJet_tau_0p5_4_WSel.png has been created
Info in <TCanvas::Print>: png file WSel/UL18/resolJet_tau_1_4_WSel.png has been created
Info in <TCanvas::Print>

### Combined UL17 + UL18

In [92]:
dict_MCscaling_UL17 = {
    'TTJets': ["722.8", 286629789811.7677, #xsdb
               '../TTJets_amcatnloFXFX-pythia8_2017UL.root',
              ],
    'TTToSemiLeptonic': ["365.34", 32392787477.907993,
                         '../TTToSemileptonic_powheg_pythia8_2017UL.root',
                        ],
    'TTTo2L2Nu': ["88.29 ", 4759777104.469413, 
                  '../TTTo2L2Nu_powheg_pythia8_2017UL.root',
                 ],
    'WJetsToLNu': ["5.368e+04", 9949891586212.455078, 
                   '../WJetsToLNu_madgraphMLM_pythia8_2017UL.root',
                  ],
    'ST_s-channel': ["3.549e+00", 70439807.475972, 
                     '../ST_s-channel_amcatnlo_pythia8_2017UL.root',
                    ],
    'ST_t-channel_top': ["1.197e+02", 656310119.274000, 
                         '../ST_t-channel_top_powheg_pythia8_2017UL.root'
                        ],
    'ST_t-channel_antitop': ["7.174e+01", 265124234.844000,  #xsdb says 82.52
                            '../ST_t-channel_antitop_powheg_pythia8_2017UL.root'
                            ],
    'ST_tW_top': ["3.245e+01", 331239851.967700, 
                  '../ST_tW_top_powheg_pythia8_2017UL.root',
                 ],
    'ST_tW_antitop': ["3.251e+01", 303960792.435600, 
                     '../ST_tW_antitop_powheg_pythia8_2017UL.root',
                     ],
    'QCD_Pt_170to300': ["7055.0", 36027673.000000, 
                        '../QCD_Muen_Pt_170to300_pythia8_2017UL.root',
                       ],
    'QCD_Pt_300to470': ["619.3", 494796.072880, 
                        '../QCD_Muen_Pt_300to470_pythia8_2017UL.root',   
                       ],
    'QCD_Pt_470to600': ["59.24", 517383.713837, 
                        '../QCD_Muen_Pt_470to600_pythia8_2017UL.root',
                       ],
    'QCD_Pt_600to800': ["18.21", 17318812.571925, 
                        '../QCD_Muen_Pt_600to800_pythia8_2017UL.root',
                       ],
    'QCD_Pt_800to1000': ["2.597e+01", 16962615.000000, 
                         '../QCD_Muen_Pt_800to1000_pythia8_2017UL.root'    
                        ],
    'QCD_Pt_1000toInf': ["1.078", 14642553.000000, 
                          '../QCD_Muen_Pt_1000toInf_pythia8_2017UL.root',
                         ],
    'WW': ["7.577e+01", 15634116.199514, 
          '../WW_pythia8_2017UL.root'  
          ],
    'WZ': ["1.21e+00", 7889000.000000, 
           '../WZ_pythia8_2017UL.root'    
          ],
    'ZZ': ["2.748e+00", 2706000.000000, 
           '../ZZ_pythia8_2017UL.root'
          ]
    }
dict_MCscaling_UL18 = {
    'TTJets': ["722.8", 297902725633.38257,
               '../TTJets_amcatnloFXFX-pythia8_2018UL.root',
              ],
    'TTToSemiLeptonic': ["365.34", 31378075292.893986,
                         '../TTToSemileptonic_powheg_pythia8_2018UL.root',
                        ],
    'TTTo2L2Nu': ["88.29 ", 5063754431.9676, 
                  '../TTTo2L2Nu_powheg_pythia8_2018UL.root',
                 ],
    'WJetsToLNu': ["5.368e+04", 10484986870276.357422, 
                   '../WJetsToLNu_madgraphMLM_pythia8_2018UL.root',
                  ],
    'ST_s-channel': ["3.549e+00", 64014403.924644, 
                     '../ST_s-channel_amcatnlo_pythia8_2018UL.root',
                    ],
    'ST_t-channel_top': ["1.197e+02", 683747124.990000, 
                         '../ST_t-channel_top_powheg_pythia8_2018UL.root'
                        ],
    'ST_t-channel_antitop': ["7.174e+01", 271219283.652000, 
                            '../ST_t-channel_antitop_powheg_pythia8_2018UL.root'
                            ],
    'ST_tW_top': ["3.245e+01", 328321574.253000, 
                  '../ST_tW_top_powheg_pythia8_2018UL.root',
                 ],
    'ST_tW_antitop': ["3.251e+01", 296655487.557600, 
                     '../ST_tW_antitop_powheg_pythia8_2018UL.root',
                     ],
    'QCD_Pt_170to300': ["7055.0", 0.,#36027673.000000, 
                        '../QCD_Muen_Pt_170to300_pythia8_2018UL.root',
                       ],
    'QCD_Pt_300to470': ["619.3", 0.,#494796.072880, 
                        '../QCD_Muen_Pt_300to470_pythia8_2018UL.root',   
                       ],
    'QCD_Pt_470to600': ["59.24", 0.,#517383.713837, 
                        '../QCD_Muen_Pt_470to600_pythia8_2018UL.root',
                       ],
    'QCD_Pt_600to800': ["18.21", 0.,#17318812.571925, 
                        '../QCD_Muen_Pt_600to800_pythia8_2018UL.root',
                       ],
    'QCD_Pt_800to1000': ["2.597e+01", 0.,#39261600.000000, 
                         '../QCD_Pt_800to1000_pythia8_2018UL.root'    
                        ],
    'QCD_Pt_1000toInf': ["1.078", 0.,#14642553.000000, 
                          '../QCD_Muen_Pt_1000toInf_pythia8_2018UL.root',
                         ],
    
    'WW': ["7.577e+01", 7959266.140092, 
          '../WW_pythia8_2018UL.root'  
          ],
    'WZ': ["1.21e+00", 3976000.000000, 
           '../WZ_pythia8_2018UL.root'    
          ],
    'ZZ': ["2.748e+00", 2000000.000000, 
           '../ZZ_pythia8_2018UL.root'
          ]
    }




In [93]:
nevts = np.array(list(float(dict_items_17[k][1][1])+float(dict_items_18[k][1][1]) for k in np.arange(0,len(dict_items_17)))) 
print (nevts)

[5.84532515e+11 6.37708628e+10 9.82353154e+09 2.04348785e+13
 1.34454211e+08 1.34005724e+09 5.36343518e+08 6.59561426e+08
 6.00616280e+08 3.60276730e+07 4.94796073e+05 5.17383714e+05
 1.73188126e+07 1.69626150e+07 1.46425530e+07 2.35933823e+07
 1.18650000e+07 4.70600000e+06]


In [109]:

dict_MCscaling_combined = {
    'TTJets': ["722.8", 5.84532515e+11,
               '../TTJets_amcatnloFXFX-pythia8_UL17-18.root',
              ],
    'TTToSemiLeptonic': ["365.34", 6.37708628e+10,
                         '../TTToSemileptonic_powheg_pythia8_UL17-18.root',
                        ],
    'TTTo2L2Nu': ["88.29 ", 9.82353154e+09, 
                  '../TTTo2L2Nu_powheg_pythia8_UL17-18.root',
                 ],
    'WJetsToLNu': ["5.368e+04", 2.04348785e+13, 
                   '../WJetsToLNu_madgraphMLM_pythia8_UL17-18.root',
                  ],
    'ST_s-channel': ["3.549e+00", 1.34454211e+08, 
                     '../ST_s-channel_amcatnlo_pythia8_UL17-18.root',
                    ],
    'ST_t-channel_top': ["1.197e+02", 1.34005724e+09, 
                         '../ST_t-channel_top_powheg_pythia8_UL17-18.root'
                        ],
    'ST_t-channel_antitop': ["7.174e+01", 5.36343518e+08, 
                            '../ST_t-channel_antitop_powheg_pythia8_UL17-18.root'
                            ],
    'ST_tW_top': ["3.245e+01", 6.59561426e+08, 
                  '../ST_tW_top_powheg_pythia8_UL17-18.root',
                 ],
    'ST_tW_antitop': ["3.251e+01", 6.00616280e+08 , 
                     '../ST_tW_antitop_powheg_pythia8_UL17-18.root',
                     ],
    'QCD_Pt_170to300': ["7055.0", 36027673.000000, 
                        '../QCD_Muen_Pt_170to300_pythia8_2017UL.root',
                       ],
    'QCD_Pt_300to470': ["619.3", 494796.072880, 
                        '../QCD_Muen_Pt_300to470_pythia8_2017UL.root',   
                       ],
    'QCD_Pt_470to600': ["59.24", 517383.713837, 
                        '../QCD_Muen_Pt_470to600_pythia8_2017UL.root',
                       ],
    'QCD_Pt_600to800': ["18.21", 17318812.571925, 
                        '../QCD_Muen_Pt_600to800_pythia8_2017UL.root',
                       ],
    'QCD_Pt_800to1000': ["2.597e+01", 39261600.000000, 
                         '../QCD_Muen_Pt_800to1000_pythia8_2017UL.root'   
                        ],
    'QCD_Pt_1000toInf': ["1.078", 14642553.000000, 
                          '../QCD_Muen_Pt_1000toInf_pythia8_2017UL.root',
                         ],
    
    'WW': ["7.577e+01", 2.35933823e+07, 
          '../WW_pythia8_UL17-18.root'  
          ],
    'WZ': ["1.21e+00", 1.18650000e+07, 
           '../WZ_pythia8_UL17-18.root'    
          ],
    'ZZ': ["2.748e+00", 4.70600000e+06, 
           '../ZZ_pythia8_UL17-18.root'
          ]
    }


In [95]:
lumi=41472.052+59816.39
print (lumi)

101288.44200000001


In [96]:
samples_17 = list(dict_MCscaling_UL17.keys())
dict_items_17 = list(dict_MCscaling_UL17.items())

samples_18 = list(dict_MCscaling_UL18.keys())
dict_items_18 = list(dict_MCscaling_UL18.items())

samples_combined = list(dict_MCscaling_combined.keys())
dict_items_combined = list(dict_MCscaling_combined.items())

print (dict_items_combined)

[('TTJets', ['722.8', 584532515000.0, '../TTJets_amcatnloFXFX-pythia8_UL17-18.root']), ('TTToSemiLeptonic', ['365.34', 63770862800.0, '../TTToSemileptonic_powheg_pythia8_UL17-18.root']), ('TTTo2L2Nu', ['88.29 ', 9823531540.0, '../TTTo2L2Nu_powheg_pythia8_UL17-18.root']), ('WJetsToLNu', ['5.368e+04', 20434878500000.0, '../WJetsToLNu_madgraphMLM_pythia8_UL17-18.root']), ('ST_s-channel', ['3.549e+00', 134454211.0, '../ST_s-channel_amcatnlo_pythia8_UL17-18.root']), ('ST_t-channel_top', ['1.197e+02', 1340057240.0, '../ST_t-channel_top_powheg_pythia8_UL17-18.root']), ('ST_t-channel_antitop', ['7.174e+01', 536343518.0, '../ST_t-channel_antitop_powheg_pythia8_UL17-18.root']), ('ST_tW_top', ['3.245e+01', 659561426.0, '../ST_tW_top_powheg_pythia8_UL17-18.root']), ('ST_tW_antitop', ['3.251e+01', 600616280.0, '../ST_tW_antitop_powheg_pythia8_UL17-18.root']), ('QCD_Pt_170to300', ['7055.0', 36027673.0, '../QCD_Muen_Pt_170to300_pythia8_2017UL.root']), ('QCD_Pt_300to470', ['619.3', 494796.07288, '../Q

In [97]:
xsecs = np.array(list(float(dict_items_combined[k][1][0]) for k in np.arange(0,len(dict_items_combined)))) 
nevts = np.array(list(float(dict_items_combined[k][1][1]) for k in np.arange(0,len(dict_items_combined)))) 
fpathsMC = np.array(list((dict_items_combined[k][1][2]) for k in np.arange(0,len(dict_items_combined))))
fpathData = '../SingleMuonAll_UL_17-18.root'
for i in range(0,len(samples_combined)):
    print (i, xsecs[i], samples_combined[i], nevts[i], fpathsMC[i])

0 722.8 TTJets 584532515000.0 ../TTJets_amcatnloFXFX-pythia8_UL17-18.root
1 365.34 TTToSemiLeptonic 63770862800.0 ../TTToSemileptonic_powheg_pythia8_UL17-18.root
2 88.29 TTTo2L2Nu 9823531540.0 ../TTTo2L2Nu_powheg_pythia8_UL17-18.root
3 53680.0 WJetsToLNu 20434878500000.0 ../WJetsToLNu_madgraphMLM_pythia8_UL17-18.root
4 3.549 ST_s-channel 134454211.0 ../ST_s-channel_amcatnlo_pythia8_UL17-18.root
5 119.7 ST_t-channel_top 1340057240.0 ../ST_t-channel_top_powheg_pythia8_UL17-18.root
6 71.74 ST_t-channel_antitop 536343518.0 ../ST_t-channel_antitop_powheg_pythia8_UL17-18.root
7 32.45 ST_tW_top 659561426.0 ../ST_tW_top_powheg_pythia8_UL17-18.root
8 32.51 ST_tW_antitop 600616280.0 ../ST_tW_antitop_powheg_pythia8_UL17-18.root
9 7055.0 QCD_Pt_170to300 36027673.0 ../QCD_Muen_Pt_170to300_pythia8_2017UL.root
10 619.3 QCD_Pt_300to470 494796.07288 ../QCD_Muen_Pt_300to470_pythia8_2017UL.root
11 59.24 QCD_Pt_470to600 517383.713837 ../QCD_Muen_Pt_470to600_pythia8_2017UL.root
12 18.21 QCD_Pt_600to800 173

In [98]:
print (len((dict_MCscaling_UL17.keys())))

18


In [99]:
import array
from array import array
def Bin(Range):
    bins=np.array([x for x in np.linspace(num=Range[0]+1,start=Range[1],stop=Range[2])])
    return bins

def getRebinnedRescaledHist(hist_name, file_name, xrange, lumi, xsec, nevt, isMC=True, rebin=True):
    #if not(sample_name in file_name): 
    #    return "File either not available or needs to be renamed to be compatible with this (poorly,) hard-coded piece of software..."
    
    FILE = ROOT.TFile.Open(file_name,'read')
    tree = FILE.Get('jetObservables')
    hist = tree.Get(hist_name)
    hist.SetDirectory(0)
    
    ROOT.TH1.AddDirectory(ROOT.kFALSE);
    
    bins = array('d', np.array(Bin(xrange)))
    
    if rebin: newHist = ROOT.TH1F(hist.Rebin(len(bins)-1, "%s_rebin"%hist_name, bins))
    else: newHist = hist.Clone()
    #newHist.SetStats(ROOT.kFALSE)
    if isMC==True: newHist.Scale(lumi*xsec/nevt)
    
    return newHist

In [100]:
Hists_WSel=np.array([
    'recoJet_pt_nom_WSel', 'recoJet_mass_nom_WSel', #'recoJetTau21_nom_WSel', 'recoJetTau32_nom_WSel',
    'recoJet_tau_0p5_1_nom_WSel', 'recoJet_tau_1_1_nom_WSel', 'recoJet_tau_2_1_nom_WSel',
    'recoJet_tau_0p5_2_nom_WSel', 'recoJet_tau_1_2_nom_WSel', 'recoJet_tau_2_2_nom_WSel',
    'recoJet_tau_0p5_3_nom_WSel', 'recoJet_tau_1_3_nom_WSel', 'recoJet_tau_2_3_nom_WSel',
    'recoJet_tau_0p5_4_nom_WSel', 'recoJet_tau_1_4_nom_WSel', 'recoJet_tau_2_4_nom_WSel',
    'nPVs_WSel','nleps_WSel',
    'muons_pt_WSel','muons_eta_WSel','muons_phi_WSel',
    'nAK8jets_WSel','AK8jets_pt_WSel','AK8jets_eta_WSel','AK8jets_phi_WSel','AK8jets_mass_WSel',
    'nAK4jets_WSel','AK4jets_pt_WSel','AK4jets_eta_WSel','AK4jets_phi_WSel','AK4jets_mass_WSel',
    'METPt_WSel','HT_WSel'])
#Objects_WSel=np.concatenate((Objects1_WSel,Objects3_WSel),axis=0)

Histnames_WSel=np.array([
    ['Events/(50 GeV)','P_{t}','P_{t}/GeV'],['Events/(10 GeV)','Softdrop Mass','Softdrop Mass/GeV'],#['Events/0.05','#tau_{21}','#tau_{21}'],['Events/0.05','#tau_{32}','#tau_{32}'],
    ['Events/0.03','#tau_{1}^{(0.5)}','#tau_{1}^{(0.5)}'],['Events/0.02','#tau_{1}^{(1)}','#tau_{1}^{(1)}'],['Events/0.02','#tau_{1}^{(2)}','#tau_{1}^{(2)}'],
    ['Events/0.03','#tau_{2}^{(0.5)}','#tau_{2}^{(0.5)}'],['Events/0.02','#tau_{2}^{(1)}','#tau_{2}^{(1)}'],['Events/0.01','#tau_{2}^{(2)}','#tau_{2}^{(2)}'],
    ['Events/0.02','#tau_{3}^{(0.5)}','#tau_{3}^{(0.5)}'],['Events/0.02','#tau_{3}^{(1)}','#tau_{3}^{(1)}'],['Events/0.01','#tau_{3}^{(2)}','#tau_{3}^{(2)}'],
    ['Events/0.02','#tau_{4}^{(0.5)}','#tau_{4}^{(0.5)}'],['Events/0.01','#tau_{4}^{(1)}','#tau_{4}^{(1)}'],['Events/0.01','#tau_{4}^{(2)}','#tau_{4}^{(2)}'],
    ['Events/2','Number of PV','Number of PV'],['Events','Number of Leptons','Number of Leptons'],
    ['Events/(20 GeV)','P_{t} of Muons','P_{t} of Muons/GeV'],['Events/0.4','#eta of Muons','#eta of Muons'],['Events/(#pi/10)','#phi of Muons','#phi of Muons'],
    ['Events','Number of AK8Jets','Number of AK8Jets'],['Events/(60 GeV)','P_{t} of AK8Jets','P_{t} of AK8Jets/GeV'],['Events/0.4','#eta of AK8Jets','#eta of AK8Jets'],['Events/(#pi/10)','#phi of AK8Jets','#phi of AK8Jets'],['Events/(10 GeV)','Mass of AK8Jets','Mass of AK8Jets/GeV'],
    ['Events','Number of AK4Jets','Number of AK4Jets'],['Events/(40 GeV)','P_{t} of AK4Jets','P_{t} of AK4Jets/GeV'],['Events/0.4','#eta of AK4Jets','#eta of AK4Jets'],['Events/(#pi/10)','#phi of AK4Jets','#phi of AK4Jets'],['Events/(20 GeV)','Mass of AK4Jets','Mass of AK4Jets/GeV'],
    ['Events/(20 GeV)','MET P_{t}','MET P_{t}/GeV'],['Events/(100 GeV)','HT','HT/GeV']])
Range_WSel=[
    [20,0,1000], [11,50,160],#[20,0,1],[20,0,1],
    [30,0,0.9],[35,0,0.7],[20,0,0.4],
    [20,0,0.6],[20,0,0.4],[20,0,0.2],
    [25,0,0.5],[20,0,0.4],[20,0,0.1],
    [20,0,0.4],[20,0,0.2],[20,0,0.1],  
    [35,0,70],[4,0,4],
    [20,0,400],[20,-4,4],[20,-3.14259, 3.14159],
    [8,0,8],[20,0,1200],[20,-4,4],[20,-3.14259, 3.14159],[15,0,150],
    [8,0,8],[20,0,800],[20,-4,4],[20,-3.14259, 3.14159],[20,0,400],
    [30,0,600],[20,0,2000]
]
print (len(Hists_WSel),len(Histnames_WSel),len(Range_WSel))

31 31 31


In [101]:
print (Histnames_WSel[0][0]
      )

Events/(50 GeV)


In [102]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.12

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [103]:
c = np.zeros(len(Range_WSel), ROOT.TCanvas)
l = np.zeros(len(Range_WSel), ROOT.TLegend)

H_ref = 720; 
W_ref = 1024; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.97
y1_l = 0.85

dx_l = 0.35
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_WSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_WSel)):
    
    print (i, Hists_WSel[i], fpathData, Range_WSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],0,0,W,H)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    c[i].SetLeftMargin( L/W )
    c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 0.95, -1,0,0);
    pad1.SetBottomMargin(0.); #// Upper and lower plot are joined
    #pad1.SetGridx();         #// Vertical grid
    pad1.SetBorderMode(0)
    pad1.SetBorderSize(0)
    pad1.SetFrameLineWidth(0)
    pad1.SetFrameBorderMode(0)
    pad1.Draw()           #// Draw the upper pad: pad1
    pad1.cd()
    Pad1.append(pad1)
    
    
    

    MC = ROOT.THStack("MC", Hists_WSel[i])
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1]))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetFillColor(ROOT.kGreen+3)
    
    TTDileptonic = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[2], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[2], nevt=nevts[2]))
    TTDileptonic.SetLineColor(ROOT.kRed-9)
    TTDileptonic.SetFillColor(ROOT.kRed-9)
    
    WJets = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[3], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[3], nevt=nevts[3]))
    WJets.SetLineColor(ROOT.kCyan-6)
    WJets.SetFillColor(ROOT.kCyan-6)
    
    SingleTop = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(4,9):
        #print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                               file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                               lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        SingleTop.Add(hist)
        
    SingleTop.SetLineColor(ROOT.kAzure+8)
    SingleTop.SetFillColor(ROOT.kAzure+8)
    
    QCD = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(9,14):
        #print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                         file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                         lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        QCD.Add(hist)
    QCD.SetLineColor(ROOT.kPink-1)
    QCD.SetFillColor(ROOT.kPink-1)
    
    VV = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(14,17):
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                        file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                        lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        VV.Add(hist)
    VV.SetLineColor(ROOT.kGray+2)
    VV.SetFillColor(ROOT.kGray+2)
    
    #VV.GetYaxis().SetRangeUser(-0.5, 1.25*TTSemileptonic.GetMaximum())
    MC.Add(VV)
    MC.Add(SingleTop)
    MC.Add(QCD)
    MC.Add(WJets)
    MC.Add(TTDileptonic)
    MC.Add(TTSemileptonic)
    dummy = MC.GetHists().At(0)
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    dummy.Draw()  
    dummy.GetYaxis().SetTitle(Histnames_WSel[i][0])
    dummy.GetXaxis().SetTitle(Histnames_WSel[i][2]) 
    print(Histnames_WSel[i][2])
    dummy.GetYaxis().SetRangeUser(-0.5, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    MC.Draw("hist e")
    
    c[i].Update()
    MC.Draw("histsame e")
    Data_SingleMuon_Combined = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathData, xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], isMC=False))
    Data_SingleMuon_Combined.SetMarkerSize(1.5)
    Data_SingleMuon_Combined.SetMarkerColor(ROOT.kBlack)
    Data_SingleMuon_Combined.SetMarkerStyle(20)
    Data_SingleMuon_Combined.SetLineColor(ROOT.kBlack)
    Data_SingleMuon_Combined.Draw("epsamex0")
    
    
    CMS_lumi.CMS_lumi(c[i], iPeriod, iPos)
    c[i].cd()
    c[i].Update()
    c[i].RedrawAxis()
    frame = c[i].GetFrame()
    frame.Draw()

    pad2 = ROOT.TPad("pad2","pad2",0,0.0,1,0.3, -1,0,0)
    pad2.SetTopMargin(0)
    pad2.SetGridx()
    pad2.SetGridy()
    pad2.Draw()
    pad2.cd()
    pad2.SetBottomMargin(0.3)
    Pad2.append(pad2)

    ratio = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    ratio.GetXaxis().SetTitle(Histnames_WSel[i][2])
    ratio.Add(Data_SingleMuon_Combined)
    ratio.GetYaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetXaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetYaxis().SetRangeUser(0.3,1.7)
    
    mc_rt = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    mc_rt.Add(TTSemileptonic)
    mc_rt.Add(TTDileptonic)
    mc_rt.Add(SingleTop)
    mc_rt.Add(QCD)
    mc_rt.Add(VV)
    mc_rt.Add(WJets)
    
    ratio.GetYaxis().SetTitle("#frac{Data}{MC}")
    ratio.Sumw2()
    ratio.SetStats(0)
    ratio.Divide(mc_rt)
    ratio.GetYaxis().SetLabelSize(0.09)
    ratio.GetXaxis().SetLabelSize(0.09)
    ratio.SetMarkerColor(ROOT.kBlack)
    ratio.SetMarkerStyle(20)
    ratio.SetLineColor(ROOT.kBlack)
    ratio.GetYaxis().SetTitleSize(0.12)
    ratio.GetYaxis().SetTitleOffset(0.4)
    ratio.GetXaxis().SetTitleSize(0.12)
    ratio.GetXaxis().SetTitleOffset(0.85)
    Ratio.append(ratio)
    #ratio.SetAxisRange(50,150)

    ratio.Draw("EP")
    c[i].cd()

    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} semileptonic (Powheg)","f")
    l[i].AddEntry(TTDileptonic,"t#bar{t} dileptonic","f")
    l[i].AddEntry(SingleTop,"Single top","f")
    l[i].AddEntry(WJets,"W+Jets","f")
    l[i].AddEntry(QCD,"QCD multijets","f")
    l[i].AddEntry(VV,"Diboson","f")
    l[i].AddEntry(Data_SingleMuon_Combined,"Data UL17+UL18","lep")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("WSel/Combined/"+Hists_WSel[i]+"_powheg_p8_UL17-18.png")
    c[i].Close()

0 recoJet_pt_nom_WSel ../SingleMuonAll_UL_17-18.root [20, 0, 1000] 101288.44200000001 365.34 63770862800.0
P_{t}/GeV
1 recoJet_mass_nom_WSel ../SingleMuonAll_UL_17-18.root [11, 50, 160] 101288.44200000001 365.34 63770862800.0
Softdrop Mass/GeV
2 recoJet_tau_0p5_1_nom_WSel ../SingleMuonAll_UL_17-18.root [30, 0, 0.9] 101288.44200000001 365.34 63770862800.0
#tau_{1}^{(0.5)}
3 recoJet_tau_1_1_nom_WSel ../SingleMuonAll_UL_17-18.root [35, 0, 0.7] 101288.44200000001 365.34 63770862800.0
#tau_{1}^{(1)}
4 recoJet_tau_2_1_nom_WSel ../SingleMuonAll_UL_17-18.root [20, 0, 0.4] 101288.44200000001 365.34 63770862800.0
#tau_{1}^{(2)}
5 recoJet_tau_0p5_2_nom_WSel ../SingleMuonAll_UL_17-18.root [20, 0, 0.6] 101288.44200000001 365.34 63770862800.0
#tau_{2}^{(0.5)}
6 recoJet_tau_1_2_nom_WSel ../SingleMuonAll_UL_17-18.root [20, 0, 0.4] 101288.44200000001 365.34 63770862800.0
#tau_{2}^{(1)}
7 recoJet_tau_2_2_nom_WSel ../SingleMuonAll_UL_17-18.root [20, 0, 0.2] 101288.44200000001 365.34 63770862800.0
#tau_{2

Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/Combined/recoJet_pt_nom_WSel_powheg_p8_UL17-18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/Combined/recoJet_mass_nom_WSel_powheg_p8_UL17-18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/Combined/recoJet_tau_0p5_1_nom_WSel_powheg_p8_UL17-18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/Combined/recoJet_tau_1_1_nom_WSel_powheg_p8_UL17-18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/Combined/recoJ

In [104]:
c = np.zeros(len(Range_WSel), ROOT.TCanvas)
l = np.zeros(len(Range_WSel), ROOT.TLegend)

H_ref = 720; 
W_ref = 1024; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.97
y1_l = 0.85

dx_l = 0.35
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_WSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_WSel)):
    
    print (i, Hists_WSel[i], fpathData, Range_WSel[i], lumi, xsecs[0], nevts[0])

    c[i] = ROOT.TCanvas(CName[i],CName[i],0,0,W,H)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    c[i].SetLeftMargin( L/W )
    c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    #hist.GetXaxis().SetRangeUser(0., 200.)
    
    pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 0.95, -1,0,0);
    pad1.SetBottomMargin(0.); #// Upper and lower plot are joined
    #pad1.SetGridx();         #// Vertical grid
    pad1.SetBorderMode(0)
    pad1.SetBorderSize(0)
    pad1.SetFrameLineWidth(0)
    pad1.SetFrameBorderMode(0)
    pad1.Draw()           #// Draw the upper pad: pad1
    pad1.cd()
    Pad1.append(pad1)
    
    
    

    MC = ROOT.THStack("MC", Hists_WSel[i])
    
    TTJets = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[0], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[0], nevt=nevts[0]))
    TTJets.SetLineColor(ROOT.kGreen+3)
    TTJets.SetFillColor(ROOT.kGreen+3)
    
    
    WJets = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[3], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[3], nevt=nevts[3]))
    WJets.SetLineColor(ROOT.kCyan-6)
    WJets.SetFillColor(ROOT.kCyan-6)
    
    SingleTop = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(4,9):
        print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                               file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                               lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        SingleTop.Add(hist)
        
    SingleTop.SetLineColor(ROOT.kAzure+8)
    SingleTop.SetFillColor(ROOT.kAzure+8)
    
    QCD = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(9,14):
        print (j,fpathsMC[j],xsecs[j],nevts[j])
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                         file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                         lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        QCD.Add(hist)
    QCD.SetLineColor(ROOT.kPink-1)
    QCD.SetFillColor(ROOT.kPink-1)
    
    VV = ROOT.TH1F("",";;",*Range_WSel[i])
    for j in range(14,17):
        hist = getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                        file_name=fpathsMC[j], xrange=Range_WSel[i], 
                                        lumi=lumi, xsec=xsecs[j], nevt=nevts[j])
        VV.Add(hist)
    VV.SetLineColor(ROOT.kGray+2)
    VV.SetFillColor(ROOT.kGray+2)
    
    #VV.GetYaxis().SetRangeUser(-0.5, 1.25*TTJets.GetMaximum())
    MC.Add(VV)
    MC.Add(SingleTop)
    MC.Add(QCD)
    MC.Add(WJets)
    MC.Add(TTJets)
    dummy = MC.GetHists().At(0)
    #dummy.SetAxisRange(0., TTJets.GetMaximum()*1.5)
    dummy.Draw()  
    dummy.GetYaxis().SetTitle(Histnames_WSel[i][0])
    dummy.GetXaxis().SetTitle(Histnames_WSel[i][2]) 
    print(Histnames_WSel[i][2])
    dummy.GetYaxis().SetRangeUser(-0.5, 2.5*TTJets.GetMaximum())
    c[i].Update()
    MC.Draw("hist e")
    
    c[i].Update()
    MC.Draw("histsame e")
    Data_SingleMuon_Combined = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathData, xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[0], nevt=nevts[0], isMC=False))
    Data_SingleMuon_Combined.SetMarkerSize(1.5)
    Data_SingleMuon_Combined.SetMarkerColor(ROOT.kBlack)
    Data_SingleMuon_Combined.SetMarkerStyle(20)
    Data_SingleMuon_Combined.SetLineColor(ROOT.kBlack)
    Data_SingleMuon_Combined.Draw("epsamex0")
    
    
    CMS_lumi.CMS_lumi(c[i], iPeriod, iPos)
    c[i].cd()
    c[i].Update()
    c[i].RedrawAxis()
    frame = c[i].GetFrame()
    frame.Draw()

    pad2 = ROOT.TPad("pad2","pad2",0,0.0,1,0.3, -1,0,0)
    pad2.SetTopMargin(0)
    pad2.SetGridx()
    pad2.SetGridy()
    pad2.Draw()
    pad2.cd()
    pad2.SetBottomMargin(0.3)
    Pad2.append(pad2)

    ratio = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    ratio.GetXaxis().SetTitle(Histnames_WSel[i][2])
    ratio.Add(Data_SingleMuon_Combined)
    ratio.GetYaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetXaxis().CenterTitle(ROOT.kTRUE);
    ratio.GetYaxis().SetRangeUser(0.3,1.7)
    
    mc_rt = ROOT.TH1D(Hists_WSel[i],";;",*Range_WSel[i])
    mc_rt.Add(TTJets)
    mc_rt.Add(SingleTop)
    mc_rt.Add(QCD)
    mc_rt.Add(VV)
    mc_rt.Add(WJets)
    
    ratio.GetYaxis().SetTitle("#frac{Data}{MC}")
    ratio.Sumw2()
    ratio.SetStats(0)
    ratio.Divide(mc_rt)
    ratio.GetYaxis().SetLabelSize(0.09)
    ratio.GetXaxis().SetLabelSize(0.09)
    ratio.SetMarkerColor(ROOT.kBlack)
    ratio.SetMarkerStyle(20)
    ratio.SetLineColor(ROOT.kBlack)
    ratio.GetYaxis().SetTitleSize(0.12)
    ratio.GetYaxis().SetTitleOffset(0.4)
    ratio.GetXaxis().SetTitleSize(0.12)
    ratio.GetXaxis().SetTitleOffset(0.85)
    Ratio.append(ratio)
    #ratio.SetAxisRange(50,150)

    ratio.Draw("EP")
    c[i].cd()

    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTJets,"t#bar{t} (aMC@NLO)","f")
    l[i].AddEntry(SingleTop,"Single top","f")
    l[i].AddEntry(WJets,"W+Jets","f")
    l[i].AddEntry(QCD,"QCD multijets","f")
    l[i].AddEntry(VV,"Diboson","f")
    l[i].AddEntry(Data_SingleMuon_Combined,"Data UL17+UL18","lep")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("WSel/Combined/"+Hists_WSel[i]+"_aMCatNLO_UL17-18.png")
    c[i].Close()

0 recoJet_pt_nom_WSel ../SingleMuonAll_UL_17-18.root [20, 0, 1000] 101288.44200000001 722.8 584532515000.0
4 ../ST_s-channel_amcatnlo_pythia8_UL17-18.root 3.549 134454211.0
5 ../ST_t-channel_top_powheg_pythia8_UL17-18.root 119.7 1340057240.0
6 ../ST_t-channel_antitop_powheg_pythia8_UL17-18.root 71.74 536343518.0
7 ../ST_tW_top_powheg_pythia8_UL17-18.root 32.45 659561426.0
8 ../ST_tW_antitop_powheg_pythia8_UL17-18.root 32.51 600616280.0
9 ../QCD_Muen_Pt_170to300_pythia8_2017UL.root 7055.0 36027673.0
10 ../QCD_Muen_Pt_300to470_pythia8_2017UL.root 619.3 494796.07288
11 ../QCD_Muen_Pt_470to600_pythia8_2017UL.root 59.24 517383.713837
12 ../QCD_Muen_Pt_600to800_pythia8_2017UL.root 18.21 17318812.571925
13 ../QCD_Muen_Pt_800to1000_pythia8_2017UL.root 25.97 39261600.0
P_{t}/GeV
1 recoJet_mass_nom_WSel ../SingleMuonAll_UL_17-18.root [11, 50, 160] 101288.44200000001 722.8 584532515000.0
4 ../ST_s-channel_amcatnlo_pythia8_UL17-18.root 3.549 134454211.0
5 ../ST_t-channel_top_powheg_pythia8_UL17-18

4 ../ST_s-channel_amcatnlo_pythia8_UL17-18.root 3.549 134454211.0
5 ../ST_t-channel_top_powheg_pythia8_UL17-18.root 119.7 1340057240.0
6 ../ST_t-channel_antitop_powheg_pythia8_UL17-18.root 71.74 536343518.0
7 ../ST_tW_top_powheg_pythia8_UL17-18.root 32.45 659561426.0
8 ../ST_tW_antitop_powheg_pythia8_UL17-18.root 32.51 600616280.0
9 ../QCD_Muen_Pt_170to300_pythia8_2017UL.root 7055.0 36027673.0
10 ../QCD_Muen_Pt_300to470_pythia8_2017UL.root 619.3 494796.07288
11 ../QCD_Muen_Pt_470to600_pythia8_2017UL.root 59.24 517383.713837
12 ../QCD_Muen_Pt_600to800_pythia8_2017UL.root 18.21 17318812.571925
13 ../QCD_Muen_Pt_800to1000_pythia8_2017UL.root 25.97 39261600.0
#tau_{4}^{(1)}
13 recoJet_tau_2_4_nom_WSel ../SingleMuonAll_UL_17-18.root [20, 0, 0.1] 101288.44200000001 722.8 584532515000.0
4 ../ST_s-channel_amcatnlo_pythia8_UL17-18.root 3.549 134454211.0
5 ../ST_t-channel_top_powheg_pythia8_UL17-18.root 119.7 1340057240.0
6 ../ST_t-channel_antitop_powheg_pythia8_UL17-18.root 71.74 536343518.0
7 

23 AK8jets_mass_WSel ../SingleMuonAll_UL_17-18.root [15, 0, 150] 101288.44200000001 722.8 584532515000.0
4 ../ST_s-channel_amcatnlo_pythia8_UL17-18.root 3.549 134454211.0
5 ../ST_t-channel_top_powheg_pythia8_UL17-18.root 119.7 1340057240.0
6 ../ST_t-channel_antitop_powheg_pythia8_UL17-18.root 71.74 536343518.0
7 ../ST_tW_top_powheg_pythia8_UL17-18.root 32.45 659561426.0
8 ../ST_tW_antitop_powheg_pythia8_UL17-18.root 32.51 600616280.0
9 ../QCD_Muen_Pt_170to300_pythia8_2017UL.root 7055.0 36027673.0
10 ../QCD_Muen_Pt_300to470_pythia8_2017UL.root 619.3 494796.07288
11 ../QCD_Muen_Pt_470to600_pythia8_2017UL.root 59.24 517383.713837
12 ../QCD_Muen_Pt_600to800_pythia8_2017UL.root 18.21 17318812.571925
13 ../QCD_Muen_Pt_800to1000_pythia8_2017UL.root 25.97 39261600.0
Mass of AK8Jets/GeV
24 nAK4jets_WSel ../SingleMuonAll_UL_17-18.root [8, 0, 8] 101288.44200000001 722.8 584532515000.0
4 ../ST_s-channel_amcatnlo_pythia8_UL17-18.root 3.549 134454211.0
5 ../ST_t-channel_top_powheg_pythia8_UL17-18.ro

Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/Combined/recoJet_pt_nom_WSel_aMCatNLO_UL17-18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/Combined/recoJet_mass_nom_WSel_aMCatNLO_UL17-18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/Combined/recoJet_tau_0p5_1_nom_WSel_aMCatNLO_UL17-18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/Combined/recoJet_tau_1_1_nom_WSel_aMCatNLO_UL17-18.png has been created
Info in <TH1F::Add>: Attempt to add histograms with different bin limits - trying to use TH1::Merge
Info in <TCanvas::Print>: png file WSel/Combined/recoJet_t

### Resolution plots


In [105]:
Hists_WSel=np.array([
    
    'resolJet_tau_0p5_1_WSel', 'resolJet_tau_1_1_WSel', 'resolJet_tau_2_1_WSel',
    'resolJet_tau_0p5_2_WSel', 'resolJet_tau_1_2_WSel', 'resolJet_tau_2_2_WSel',
    'resolJet_tau_0p5_3_WSel', 'resolJet_tau_1_3_WSel', 'resolJet_tau_2_3_WSel',
    'resolJet_tau_0p5_4_WSel', 'resolJet_tau_1_4_WSel', 'resolJet_tau_2_4_WSel',
    'resolJet_tau21_WSel', 'resolJet_tau32_WSel',
])
#Objects_WSel=np.concatenate((Objects1_WSel,Objects3_WSel),axis=0)

Histnames_WSel=np.array([
    ['Events','#resol tau_{1}^{(0.5)}','resol #tau_{1}^{(0.5)}'],['Events','resol #tau_{1}^{(1)}','resol #tau_{1}^{(1)}'],['Events','resol #tau_{1}^{(2)}','resol #tau_{1}^{(2)}'],
    ['Events','#resol tau_{2}^{(0.5)}','resol #tau_{2}^{(0.5)}'],['Events','resol #tau_{2}^{(1)}','resol #tau_{2}^{(1)}'],['Events','resol #tau_{2}^{(2)}','resol #tau_{2}^{(2)}'],
    ['Events','#resol tau_{3}^{(0.5)}','resol #tau_{3}^{(0.5)}'],['Events','resol #tau_{3}^{(1)}','resol #tau_{3}^{(1)}'],['Events','resol #tau_{3}^{(2)}','resol #tau_{3}^{(2)}'],
    ['Events','#resol tau_{4}^{(0.5)}','resol #tau_{4}^{(0.5)}'],['Events','resol #tau_{4}^{(1)}','resol #tau_{4}^{(1)}'],['Events','resol #tau_{4}^{(2)}','resol #tau_{4}^{(2)}'],
    ['Events','resol #tau_{2,1}^{(1)}','resol #tau_{2,1}^{(1)}'],['Events','resol #tau_{3,2}^{(1)}','resol #tau_{3,2}^{(1)}'],
    ])
Range_WSel=[
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.],[50,0.,2.],
    [50,0.,2.],[50,0.,2.]
    
    
]
print (len(Hists_WSel),len(Histnames_WSel),len(Range_WSel))

14 14 14


In [106]:
print (Histnames_WSel[0][0]
      )

Events


In [107]:
import sys
#print (sys.path)
sys.path.insert(0,'../../../jetObservables/Unfolding/python/')
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()

CMS_lumi.lumi_7TeV = "4.8 fb^{-1}"
CMS_lumi.lumi_8TeV = "18.3 fb^{-1}"
CMS_lumi.writeExtraText = True
#CMS_lumi.extraText = "Preliminary"
CMS_lumi.lumi_13TeV = "%.3f fb^{-1} (13 TeV)"%np.round(lumi/1000.0,3)
CMS_lumi.lumi_sqrtS = "%.2f fb^{-1} (13 TeV)"%np.round(lumi/1000,3)
CMS_lumi.relPosX = 0.12

iPos = 0
#if( iPos==0 ): CMS_lumi.relPosX = 0.16


In [108]:
c = np.zeros(len(Range_WSel), ROOT.TCanvas)
l = np.zeros(len(Range_WSel), ROOT.TLegend)

H_ref = 720; 
W_ref = 1024; 
W = W_ref
H  = H_ref

iPeriod = 4

# references for T, B, L, R
T = 0.08*H_ref
B = 0.12*H_ref 
L = 0.12*W_ref
R = 0.04*W_ref

x1_l = 0.97
y1_l = 0.85

dx_l = 0.3
dy_l = 0.15
x0_l = x1_l-dx_l
y0_l = y1_l-dy_l


CName=[]
for i in range(1,len(Range_WSel)+1):
    CName.append('c'+str(i))
Pad1=[]
Pad2=[]
Mc=[]
Ratio=[]

for i in range(0,len(Range_WSel)):
    
    print (i, Hists_WSel[i], fpathData, Range_WSel[i], lumi, xsecs[1], nevts[1])

    c[i] = ROOT.TCanvas(CName[i],CName[i],0,0,W,H)
    c[i].SetFillColor(0)
    c[i].SetBorderMode(0)
    c[i].SetBorderSize(0)
    c[i].SetFrameFillStyle(0)
    c[i].SetFrameBorderMode(0)
    c[i].SetFrameLineWidth(0)
    c[i].SetLeftMargin( L/W )
    c[i].SetRightMargin( R/W )
    c[i].SetTopMargin( T/H )
    c[i].SetBottomMargin( B/H )
    c[i].SetTickx(0)
    c[i].SetTicky(0)
    
    TTSemileptonic = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[1], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[1], nevt=nevts[1], rebin=False))
    TTSemileptonic.SetLineColor(ROOT.kGreen+3)
    TTSemileptonic.SetLineWidth(1)
    TTSemileptonic.SetMarkerSize(1)
    TTSemileptonic.SetMarkerStyle(26)
    TTSemileptonic.SetMarkerColor(ROOT.kGreen+3)
    TTSemileptonic.SetFillStyle(0)
    
    TTJets = (getRebinnedRescaledHist(hist_name=Hists_WSel[i], 
                                                file_name=fpathsMC[0], xrange=Range_WSel[i], 
                                                lumi=lumi, xsec=xsecs[0], nevt=nevts[0], rebin=False))
    TTJets.SetLineColor(ROOT.kRed+5)
    TTJets.SetLineWidth(1)
    TTJets.SetMarkerSize(1)
    TTJets.SetMarkerStyle(26)
    TTJets.SetMarkerColor(ROOT.kRed+5)
    TTJets.SetFillStyle(0)
    
    
    #dummy.SetAxisRange(0., TTSemileptonic.GetMaximum()*1.5)
    TTSemileptonic.GetYaxis().SetTitle(Histnames_WSel[i][0])
    TTSemileptonic.GetXaxis().SetTitle(Histnames_WSel[i][2]) 
    print(Histnames_WSel[i][2])
    #TTSemileptonic.GetYaxis().SetRangeUser(-0.5, 2.5*TTSemileptonic.GetMaximum())
    c[i].Update()
    TTSemileptonic.Rebin(2)
    TTJets.Rebin(2)
    TTSemileptonic.Draw("ep")
    TTJets.Draw("ep samee")
    
    
    l[i] =  ROOT.TLegend(x0_l,y0_l,x1_l, y1_l )
    l[i].AddEntry(TTSemileptonic,"t#bar{t} semileptonic (Powheg)","p")
    l[i].AddEntry(TTJets,"t#bar{t} inclusive (aMC@NLO)","p")
    l[i].SetTextFont(42)
    l[i].SetTextAngle(0)
    l[i].SetTextColor(ROOT.kBlack)    
    #legend.SetTextSize(0.05)    
    l[i].SetTextAlign(12) 
    l[i].SetBorderSize(0)
    l[i].SetFillStyle(0)
    #legend.SetLinewdith(0)
    l[i].Draw()

    c[i].Update()
    c[i].Draw()
    c[i].Print("WSel/Combined/"+Hists_WSel[i]+"UL17-18.png")
    c[i].Close()

0 resolJet_tau_0p5_1_WSel ../SingleMuonAll_UL_17-18.root [50, 0.0, 2.0] 101288.44200000001 365.34 63770862800.0
resol #tau_{1}^{(0.5)}
1 resolJet_tau_1_1_WSel ../SingleMuonAll_UL_17-18.root [50, 0.0, 2.0] 101288.44200000001 365.34 63770862800.0
resol #tau_{1}^{(1)}
2 resolJet_tau_2_1_WSel ../SingleMuonAll_UL_17-18.root [50, 0.0, 2.0] 101288.44200000001 365.34 63770862800.0
resol #tau_{1}^{(2)}
3 resolJet_tau_0p5_2_WSel ../SingleMuonAll_UL_17-18.root [50, 0.0, 2.0] 101288.44200000001 365.34 63770862800.0
resol #tau_{2}^{(0.5)}
4 resolJet_tau_1_2_WSel ../SingleMuonAll_UL_17-18.root [50, 0.0, 2.0] 101288.44200000001 365.34 63770862800.0
resol #tau_{2}^{(1)}
5 resolJet_tau_2_2_WSel ../SingleMuonAll_UL_17-18.root [50, 0.0, 2.0] 101288.44200000001 365.34 63770862800.0
resol #tau_{2}^{(2)}
6 resolJet_tau_0p5_3_WSel ../SingleMuonAll_UL_17-18.root [50, 0.0, 2.0] 101288.44200000001 365.34 63770862800.0
resol #tau_{3}^{(0.5)}
7 resolJet_tau_1_3_WSel ../SingleMuonAll_UL_17-18.root [50, 0.0, 2.0] 1

Info in <TCanvas::Print>: png file WSel/Combined/resolJet_tau_0p5_1_WSelUL17-18.png has been created
Info in <TCanvas::Print>: png file WSel/Combined/resolJet_tau_1_1_WSelUL17-18.png has been created
Info in <TCanvas::Print>: png file WSel/Combined/resolJet_tau_2_1_WSelUL17-18.png has been created
Info in <TCanvas::Print>: png file WSel/Combined/resolJet_tau_0p5_2_WSelUL17-18.png has been created
Info in <TCanvas::Print>: png file WSel/Combined/resolJet_tau_1_2_WSelUL17-18.png has been created
Info in <TCanvas::Print>: png file WSel/Combined/resolJet_tau_2_2_WSelUL17-18.png has been created
Info in <TCanvas::Print>: png file WSel/Combined/resolJet_tau_0p5_3_WSelUL17-18.png has been created
Info in <TCanvas::Print>: png file WSel/Combined/resolJet_tau_1_3_WSelUL17-18.png has been created
Info in <TCanvas::Print>: png file WSel/Combined/resolJet_tau_2_3_WSelUL17-18.png has been created
Info in <TCanvas::Print>: png file WSel/Combined/resolJet_tau_0p5_4_WSelUL17-18.png has been created
In